In [237]:
import xlwings as xw
import numpy as np
import pandas as pd
from collections import OrderedDict
from time import time

'''

3. Directory and file path
4. Traverse for all bhavcopies in folder
5. Use USDINR file
6. Check overall in reports (it looks like string)
7. Report file names in seperate directory.
8. Outputs in seperate folder
'''
master_dir = 'X:\\Data Analytics\\ETL\\Master\\'
download_dir = 'X:\\Data Analytics\\ETL\\Download\\'

start_time = time()

thisdir = "D:\\Backup\\"
for r, d, f in os.walk(thisdir):
    for file in f:
        if "nsecm.txt" in file:
            print(os.path.join(r, file))



# read bhav copy as workbook

bhav_copy_file_name = master_dir
wb = xw.Book(r'C:\Users\krishnay\Desktop\RollOver\fo22OCT2018bhav.csv')

# read sheet

sht = wb.sheets['fo22OCT2018bhav']

# convert sheet to pandas dataframe

all_stocks = sht.range('A1').options(pd.DataFrame, expand='table').value

# select only FUTSTK and FUTIDX

new_index = ['FUTSTK', 'FUTIDX']
all_stocks = all_stocks.loc[new_index]

# Categorization based on master document
# read master doc


mast_book = xw.Book(r'C:\Users\krishnay\Desktop\RollOver\MasterData.xlsx')

# read sheet

mast_sheet = mast_book.sheets('Sheet1')

# convert sheet to pandas dataframe

master = mast_sheet.range('A1').options(pd.DataFrame, expand='table').value

master.rename(columns={'SymbolName': 'SYMBOL'}, inplace=True)

# filter master for symbol, bloomcode and sector

master.set_index(keys='SYMBOL', inplace=True)
master.drop(columns=['IsActiveFNO'], axis=1, inplace=True)

# join dataframes master and all_stocks based on SYMBOL

all_stocks = all_stocks.join(master, on='SYMBOL')

# create Pivot table

all_stocks = all_stocks.pivot_table(values=['OPEN_INT', 'SETTLE_PR'], index=['SYMBOL', 'Sector', 'IsNifty', 'Type'],
                                    columns='EXPIRY_DT', aggfunc={'SETTLE_PR': np.sum,
                                                                  'OPEN_INT': np.sum})

# calculate product-near and product-far

data = pd.DataFrame([map(round, all_stocks['OPEN_INT'].iloc[:, 0] * all_stocks['SETTLE_PR'].iloc[:, 0])
                        , map(round, all_stocks['OPEN_INT'].iloc[:, 1] * all_stocks['SETTLE_PR'].iloc[:, 1] +
                              all_stocks['OPEN_INT'].iloc[:, 2] * all_stocks['SETTLE_PR'].iloc[:, 2])]).T

data.columns = ['Product-Near', 'Product-Far']
data.index = all_stocks.index

pd.set_option('display.precision', 7)

Overall = pd.DataFrame({'Product-Near': data['Product-Near'].sum(),
                        'Product-Far': data['Product-Far'].sum(),
                        'Roll': round(
                            data['Product-Far'].sum() / (data['Product-Near'].sum() + data['Product-Far'].sum()), 2)},
                       index=['Overall'])


# calculate SSF roll over and convert it to %

def ssf_calculator(a, b):
    try:
        return round(b / (a + b), 3)
    except:
        return 0.0

    # apply roll over to all stocks and append ssf_rollover


data['SSF_rollover'] = data.apply(lambda row: ssf_calculator(row['Product-Near'], row['Product-Far']), axis=1)

pd.set_option('display.precision', 10)


# Group all sectotrs and run sum() aggregate

Sector = data.groupby('Sector').sum()
Sector['Roll'] = Sector.apply(lambda row: ssf_calculator(row['Product-Near'], row['Product-Far']), axis=1)

# append nifty stocks to Sector table

nifty_product_near = data.groupby('IsNifty').get_group(True).sum()['Product-Near']
nifty_product_far = data.groupby('IsNifty').get_group(True).sum()['Product-Far']
nifty_roll = ssf_calculator(nifty_product_near, nifty_product_far)

Nifty_Stocks_df = pd.DataFrame(
    {'Product-Near': [nifty_product_near], 'Product-Far': [nifty_product_far], 'Roll': [nifty_roll]},
    index=['Nifty Stocks'])

# append non nifty stocks to sector table

non_nifty_product_near = data.groupby('IsNifty').get_group(False).groupby('Type').sum()['Product-Near']['SSF']
non_nifty_product_far = data.groupby('IsNifty').get_group(False).groupby('Type').sum()['Product-Far']['SSF']
non_nifty_roll = ssf_calculator(non_nifty_product_near, non_nifty_product_far)

Non_Nifty_Stocks_df = pd.DataFrame(
    {'Product-Near': [non_nifty_product_near], 'Product-Far': [non_nifty_product_far], 'Roll': [non_nifty_roll]},
    index=['Non-Nifty Stocks'])

# append SSF stock to sector table

ssf_df = pd.DataFrame({'Product-Near': data.groupby('Type').sum().loc['SSF']['Product-Near'],
                       'Product-Far': data.groupby('Type').sum().loc['SSF']['Product-Far'],
                       'Roll': ssf_calculator(data.groupby('Type').sum().loc['SSF']['Product-Near'],
                                              data.groupby('Type').sum().loc['SSF']['Product-Far'])},
                      index=['SSF'])

# append NIFTY to sector table

NIFTY_df = pd.DataFrame({'Product-Near': data.loc['NIFTY'].loc[:, 'Product-Near'][0],
                         'Product-Far': data.loc['NIFTY'].loc[:, 'Product-Far'][0],
                         'Roll': ssf_calculator(data.loc['NIFTY'].loc[:, 'Product-Near'][0],
                                                data.loc['NIFTY'].loc[:, 'Product-Far'][0])},
                        index=['Nifty'])

# append BANKNIFTY to sector table

BANKNIFTY_df = pd.DataFrame({'Product-Near': data.loc['BANKNIFTY'].loc[:, 'Product-Near'][0],
                             'Product-Far': data.loc['BANKNIFTY'].loc[:, 'Product-Far'][0],
                             'Roll': ssf_calculator(data.loc['BANKNIFTY'].loc[:, 'Product-Near'][0],
                                                    data.loc['BANKNIFTY'].loc[:, 'Product-Far'][0])},
                            index=['NiftyBank'])

frames = [Sector, Nifty_Stocks_df, Non_Nifty_Stocks_df, ssf_df, NIFTY_df, BANKNIFTY_df, Overall]
result = pd.concat(frames, sort=False)
result.sort_index(inplace=True)
result.drop(index=['None', 'Index'], axis=0, inplace=True)

# write dataframe to excel

writer = pd.ExcelWriter('Report_stats.xlsx', engine='xlsxwriter')
result.to_excel(writer, sheet_name='SSF & Indices Report')


# Dump rollover data for expiry

def rollover_data_dumper(expiry, month, year):
    print 'Dumping data for {} {} {}.....'.format(expiry, month, year)

    # ignore index i.e consider only SSF stocks

    ssf_rollover = data.groupby('Type').get_group('SSF').reset_index(level=[1, 2, 3], drop=True)[['SSF_rollover']]

    # fetch bloomcode from master and join dataframes to dumper

    stock_bloom = master.groupby('Type').get_group('SSF')[['BloomCode']]
    dumper = stock_bloom.merge(ssf_rollover, on='SYMBOL')

    # dump expiry, month and year

    dumper['Expiry'], dumper['Month'], dumper['Year'] = expiry, month, year
    dumper.reset_index(inplace=True)

    # reaarange according to schema

    dumper = dumper[['Expiry', 'SYMBOL', 'BloomCode', 'Month', 'Year', 'SSF_rollover']]
    writer = pd.ExcelWriter('Report.xlsx', engine='xlsxwriter')
    dumper.to_excel(writer, sheet_name='SSF', index=False)

    # dump indices rollover

    indices_rollover = pd.DataFrame(result['Roll'])
    indices_rollover['Expiry'], indices_rollover['Month'], indices_rollover['Year'] = expiry, month, year
    indices_rollover.index.name = 'Indices'
    indices_rollover.reset_index(inplace=True)
    indices_rollover = indices_rollover[['Expiry', 'Month', 'Year', 'Indices', 'Roll']]
    indices_rollover.to_excel(writer, sheet_name='Indices', index=False)

    # Unrolled data

    unrolled = master.groupby('Type').get_group('SSF')[['BloomCode']].merge(
        data.groupby('Type').get_group('SSF').reset_index(
            level=[1, 2, 3], drop=True)[['Product-Near']], on='SYMBOL')

    # Convert product-near to million dollars rates
    
    dollar = pd.read_excel(master_dir+'\USDINRRates.xlsx')
    # convert string objects to datetime type 

  


    
    
    
    
    
    
    
    
    dollar = pd.read_excel(master_dir+'USDINRRates.xlsx')
    dollar
    unrolled['Million_dollar'] = unrolled['Product-Near'].apply(lambda x: round(x / (dollar * 1000000), 1))

    # dump million dollar rates of stocks

    unrolled_dumper = master.groupby('Type').get_group('SSF')[['BloomCode']]
    unrolled_dumper['Expiry'], unrolled_dumper['Month'], unrolled_dumper['Year'] = expiry, month, year
    unrolled_dumper['Million_dollar'] = unrolled['Million_dollar']
    unrolled_dumper.reset_index(inplace=True)
    unrolled_dumper = unrolled_dumper[['Expiry', 'SYMBOL', 'BloomCode', 'Month', 'Year', 'Million_dollar']]

    unrolled_dumper.to_excel(writer, sheet_name='Unrolled', index=False)


# extract date from input bhavcopy name
file_name = 'fo22OCT2018bhav.csv'[2:11]
date, month, year = file_name[:2], file_name[2:5], file_name[5:]

# read expiry from master dump to get expiry code
dates = pd.read_excel('C:/Users/krishnay/Desktop/RollOver/Expiry_dates_master.xlsx')

expiry = str(dates[(dates['Date'] == int(date))
                   & (dates['Month'] == month) & (dates['Year'] == int(year))]['Expiry'].values[0])

rollover_data_dumper(expiry, month, year)


# Calculate avergae of last three months

def average_calculator(expiry, value, stock_names, data_df):
    average = []
    for i in range(len(stock_names)):
        # average.append( round(rolldata.groupby('Expiry').get_group(expiry).loc[stock_names[i]][-3:]['SSF_rollover'].mean(), 2) )
        average.append(round(data_df.groupby('Expiry').get_group(expiry).loc[stock_names[i]][-3:][value].mean(), 2))
    return average


# rolldata.groupby('Expiry').get_group('E-5').loc['ACC'][-4:-1]['SSF_rollover'].mean()

# read from dumper object for SSF rolldata

rolldata = pd.read_excel('C:/Users/krishnay/Desktop/RollOver/RollData.xlsx')
rolldata = rolldata.set_index('SYMBOL')

# average is appended in sorted manner
stock_names = list(sorted(set(rolldata.index)))
average_ssf = pd.DataFrame(average_calculator(expiry, 'SSF_rollover', stock_names, rolldata), index=stock_names,
                           columns=['Average'])
average_ssf.index.name = 'SYMBOL'

# Greater than average
# read from dumped data


grtthanaverage = pd.read_excel('X:\Data Analytics\ETL\Report.xlsx', sheet_name='SSF')
grtthanaverage = grtthanaverage.merge(average_ssf, on='SYMBOL')

# difference of today's rollover and average of last 3 months
grtthanaverage['Difference'] = grtthanaverage['SSF_rollover'] - grtthanaverage['Average']

# read master for fetching type of stock nifty, non-nifty and physical

type_of_stock = pd.read_excel('C:/Users/krishnay/Desktop/RollOver/MasterData.xlsx', sheet_name='Physical')

grtthanaverage = grtthanaverage.merge(type_of_stock, on='SYMBOL')


# Sort elements based on criteria top or low performing stocks

def sort_stocks(rank_order, sorting_type, data, criteria):
    sorted_stocks = {}

    for group_name, group_elements in data.groupby('TYPE'):
        sorted_stocks[group_name] = group_elements.sort_values(ascending=sorting_type, by=criteria)[: rank_order]

    return sorted_stocks


# pass False for top performing as sorting parameter

grt_avg_top_stocks = sort_stocks(5, False, grtthanaverage, 'Difference')

# pass True for low performing as sorting parameter

grt_avg_low_stocks = sort_stocks(5, True, grtthanaverage, 'Difference')
# unrolled OI calculator

unrolled_today = pd.read_excel('C:/Users/krishnay/Desktop/RollOver/Unrolled_dumped.xlsx')
unrolled_today = unrolled_today.set_index('SYMBOL')

# average is appended in sorted manner
stock_names = list(sorted(set(unrolled_today.index)))

average_unrolled = pd.DataFrame(average_calculator(expiry, 'Million_dollar', stock_names, unrolled_today),
                                index=stock_names, columns=['Average'])
average_unrolled.index.name = 'SYMBOL'

# read unrolled from dumped data

unrolled_OI_calc = pd.read_excel('X:\Data Analytics\ETL\Report.xlsx', sheet_name='Unrolled')
unrolled_OI_calc = unrolled_OI_calc.merge(average_unrolled, on='SYMBOL')

# difference of today's million_unrolled and average of last 3 months
unrolled_OI_calc['Difference'] = unrolled_OI_calc['Million_dollar'] - unrolled_OI_calc['Average']

# Calculate percent for unrolled- avg/ avg
unrolled_OI_calc['Percent'] = unrolled_OI_calc.apply(
    lambda row: round((row['Million_dollar'] - row['Average']) / row['Average'] * 100), axis=1)

# read del quantity

del_qty = pd.read_excel('C:/Users/krishnay/Desktop/RollOver/Del Qty.xlsx')
del_qty = del_qty[['Del Avg']]
del_qty.index.name = 'SYMBOL'

unrolled_OI_calc = unrolled_OI_calc.merge(del_qty, on='SYMBOL')
unrolled_OI_calc['Ratio'] = unrolled_OI_calc['Million_dollar'] / unrolled_OI_calc['Del Avg']
unrolled_OI_calc['Ratio'] = unrolled_OI_calc['Ratio'].apply(lambda x: round(x))

# read master for fetching type of stock nifty, non-nifty and physical

type_of_stock = pd.read_excel('C:/Users/krishnay/Desktop/RollOver/MasterData.xlsx', sheet_name='Physical')
unrolled_OI_calc = unrolled_OI_calc.merge(type_of_stock, on='SYMBOL')

# pass False for top performing as sorting parameter (take only top stocks for unrolled)

unrolled_OI_top_stocks = sort_stocks(5, False, unrolled_OI_calc, 'Ratio')

# increased Activity
# read dumped value for SSF stucks for current expiry

increased_activity = pd.read_excel('X:\Data Analytics\ETL\Report.xlsx', sheet_name='SSF')
increased_activity.set_index('SYMBOL', inplace=True)
# read SSF stocks for T-1 expiry
expiry_list = 'E-5 E-4 E-3 E-2 E-1 E'.split(' ')

previous_day_expiry = pd.read_excel('C:/Users/krishnay/Desktop/RollOver/RollData.xlsx')
previous_day_expiry = previous_day_expiry[
    (previous_day_expiry['Expiry'] == expiry_list[expiry_list.index(expiry) - 1]) & (
                previous_day_expiry['Month'] == month) &
    (previous_day_expiry['Year'] == int(year))][['SYMBOL', 'SSF_rollover']]
previous_day_expiry.set_index('SYMBOL', inplace=True)

# merge on symbol

increased_activity = increased_activity.merge(previous_day_expiry, on='SYMBOL')
increased_activity.columns = ['Expiry', 'BloomCode', 'Month', 'Year', 'SSF_rollover_T', 'SSF_rollover_T-1']

# calculate difference Today - (T-1)

increased_activity['Difference'] = increased_activity['SSF_rollover_T'] - increased_activity['SSF_rollover_T-1']
increased_activity = increased_activity.sort_values(ascending=False, by='Difference').head(10)

dumped_indices = pd.read_excel('C:/Users/krishnay/Desktop/RollOver/Indices_dump.xlsx')
dumped_indices = dumped_indices.groupby('Expiry').get_group(expiry)
indices_names = list(sorted(set(dumped_indices['Indices'])))
dumped_indices.set_index('Indices', inplace=True)

# Average of indicies for last 3 months
avg_indices = pd.DataFrame(average_calculator(expiry, 'Roll', indices_names, dumped_indices),
                           index=indices_names, columns=['Average'])
avg_indices.index.name = 'Indices'

indices_read = pd.read_excel('X:\Data Analytics\ETL\Report.xlsx', sheet_name='Indices')

indices_diff = indices_read.set_index('Indices')
indices_diff = indices_diff.merge(avg_indices, on='Indices')
indices_diff['Difference'] = indices_diff['Roll'] - indices_diff['Average']
exclude_list = ['Nifty', 'NiftyBank', 'Nifty Stocks', 'Non-Nifty Stocks', 'SSF', 'Overall', 'Real Estate',
                'Diversified', 'chemicals', 'Media']
new_indices = indices_diff.drop(index=exclude_list)
lagging_sectors = str(new_indices.sort_values('Difference')[:2].index[0]), str(
    new_indices.sort_values('Difference')[:2].index[1])
leading_sectors = str(new_indices.sort_values('Difference', ascending=False)[:2].index[0]), str(
    new_indices.sort_values('Difference')[:2].index[1])

print_data = []

print_data.append('----- Highlights at {0} -----'.format(expiry))

for name in ['Nifty', 'NiftyBank', 'SSF', 'Nifty Stocks', 'Overall']:
    value = indices_read[indices_read['Indices'] == name]['Roll'].values[0]
    avg = avg_indices.loc[name].values[0]
    diff = value - avg
    if diff > 0:
        print_data.append(
            ' *{0} rolls ({1}%) are {2} percentage points higher than the 3 month average of {3}%. '.format(
                name, value * 100, diff * 100, avg * 100))
    else:
        print_data.append(
            ' *{0} rolls ({1}%) are {2} percentage points lower than the 3 month average of {3}%. '.format(
                name, value * 100, diff * -100, avg * 100))

print_data.append(' *Increased roll activity was seen in {0} '.format(
    ', '.join(str(x) for x in increased_activity['BloomCode'].values)))

print_data.append(
    '*Rolls in {0} and {1} names are leading their 3 month average, while those in {2} and {3} are lagging'.format(
        leading_sectors[0],
        leading_sectors[1],
        lagging_sectors[0],
        lagging_sectors[1]))
print_data.append(' *Stocks with rollovers higher than their 3 month average: ')
i = 1

data = OrderedDict(reversed(list(grt_avg_top_stocks.items())))
while True:
    for key, values in data.iteritems():
        print_data.append(' {0} '.format(key))
        stocks = []
        for name in values['BloomCode'].values:
            stocks.append("{0}({1}%)".format(str(name),
                                             int(round(
                                                 values.set_index('BloomCode').loc[name][['SSF_rollover']][0] * 100))))
        print_data.append(', '.join(stocks))
    i += 1
    if i > 2:
        break
    data = OrderedDict(reversed(list(grt_avg_low_stocks.items())))
    print_data.append(' *Stocks with rollovers lower than their 3 month average: ')

print_data.append(
    ' *SSF with high unrolled OI compared to 3 mn avg: (OI in $mn and Ratio of OI to 3mn avg of expiry day delivery )')

for key, values in OrderedDict(reversed(list(unrolled_OI_top_stocks.items()))).iteritems():
    print_data.append(' {0} '.format(key))
    stocks = []
    for name in values['BloomCode']:
        stocks.append(
            '{0}({1}/{2}x)'.format(name, values.set_index('BloomCode').loc[name]['Million_dollar'],
                                   values.set_index('BloomCode').loc[name]['Ratio']))
    print_data.append(', '.join(stocks))

pd.DataFrame(print_data).to_excel('Final.xlsx', sheet_name='Report', header=None, index=None)

end_time = time()

print "Execution time: ", end_time - start_time

C:\ProgramData\Anaconda2\lib\site-packages\ipykernel_launcher.py:80: RuntimeWarning: invalid value encountered in double_scalars


Dumping data for E-3 OCT 2018.....


In [326]:
master_dir = 'X:\Data Analytics\ETL\Master'
dollar = pd.read_excel(master_dir+'\USDINRRates.xlsx')



In [327]:
dollar_value

73.5

In [336]:
import os
download_dir = 'X:\Data Analytics\ETL\Download'

for r, d, f in os.walk(download_dir):
    for csv_file in f:
        file_name = csv_file
        
    


In [ ]:
def bhavcopy_processor(file_name):
    '''Function to read and process bhavcopy 
    Returns a pivot_table'''
    
    # read bhav copy as workbook    
    wb = xw.Book(download_dir+file_name)
    
    # read sheet
    sht = wb.sheets[file_name]
    
    # convert sheet to pandas dataframe
    all_stocks = sht.range('A1').options(pd.DataFrame, expand='table').value
    
    # select only FUTSTK and FUTIDX
    new_index = ['FUTSTK', 'FUTIDX']
    all_stocks = all_stocks.loc[new_index]

    # Categorization based on master document
    # read master doc
    mast_book = xw.Book(master_dir+'MasterData.xlsx')
    
    # read sheet
    mast_sheet = mast_book.sheets('Sheet1')

    # convert sheet to pandas dataframe
    master = mast_sheet.range('A1').options(pd.DataFrame, expand='table').value
    #master.rename(columns={'SymbolName': 'SYMBOL'}, inplace=True)

    # filter master for symbol, bloomcode and sector
    master.set_index(keys='SYMBOL', inplace=True)
    master.drop(columns=['IsActiveFNO'], axis=1, inplace=True)

    # join dataframes master and all_stocks based on SYMBOL
    all_stocks = all_stocks.join(master, on='SYMBOL')

    # create Pivot table
    all_stocks = all_stocks.pivot_table(values=['OPEN_INT', 'SETTLE_PR'], index=['SYMBOL', 'Sector', 'IsNifty', 'Type'],
                                        columns='EXPIRY_DT', aggfunc={'SETTLE_PR': np.sum,
                                                                      'OPEN_INT': np.sum})
    return all_stocks


In [363]:
abcd = ssf_calculator(data['Product-Near'][12], data['Product-Far'][12])

In [342]:
data = pd.DataFrame([map(round, all_stocks['OPEN_INT'].iloc[:, 0] * all_stocks['SETTLE_PR'].iloc[:, 0])
                        , map(round, all_stocks['OPEN_INT'].iloc[:, 1] * all_stocks['SETTLE_PR'].iloc[:, 1] +
                              all_stocks['OPEN_INT'].iloc[:, 2] * all_stocks['SETTLE_PR'].iloc[:, 2])]).T

data.columns = ['Product-Near', 'Product-Far']
data.index = all_stocks.index

In [362]:
def ssf_calculator(a, b):
    try:
        return round(b / (a + b), 3)
    except:
        return 0.0

In [364]:
abcd

0.083

In [353]:
def ssf_calculator(a, b):
    try:
        return "{:.2}".format(b / (a + b))
    except:
        return 0.0

In [399]:
import xlwings as xw
import numpy as np
import pandas as pd
from collections import OrderedDict
from time import time
from dateutil import parser
import os

start_time = time()
'''

3. Directory and file path
4. Traverse for all bhavcopies in folder
5. Use USDINR file
6. Check overall in reports (it looks like string)
7. Report file names in seperate directory.
8. Outputs in seperate folder
'''
master_dir = 'X:\\Data Analytics\\ETL\\Master\\'
download_dir = 'X:\\Data Analytics\\ETL\\Download\\'
output_dir = 'X:\\Data Analytics\\ETL\\Output\\'


def bhavcopy_processor(file_name):
    '''Function to read and process bhavcopy
    Convert it into a pivot_table
    calculate product-far/ product-near and return that dataframe '''

    # read bhav copy as workbook
    wb = xw.Book(download_dir + file_name)

    # read sheet
    sht = wb.sheets[file_name[:-4]]

    # convert sheet to pandas dataframe
    all_stocks = sht.range('A1').options(pd.DataFrame, expand='table').value

    # select only FUTSTK and FUTIDX
    new_index = ['FUTSTK', 'FUTIDX']
    all_stocks = all_stocks.loc[new_index]

    # Categorization based on master document
    # read master doc
    mast_book = xw.Book(master_dir + 'MasterData.xlsx')

    # read sheet
    mast_sheet = mast_book.sheets('Sheet1')

    # convert sheet to pandas dataframe
    master = mast_sheet.range('A1').options(pd.DataFrame, expand='table').value
    # master.rename(columns={'SymbolName': 'SYMBOL'}, inplace=True)

    # filter master for symbol, bloomcode and sector
    master.set_index(keys='SYMBOL', inplace=True)
    master.drop(columns=['IsActiveFNO'], axis=1, inplace=True)

    # join dataframes master and all_stocks based on SYMBOL
    all_stocks = all_stocks.join(master, on='SYMBOL')

    # create Pivot table
    all_stocks = all_stocks.pivot_table(values=['OPEN_INT', 'SETTLE_PR'], index=['SYMBOL', 'Sector', 'IsNifty', 'Type'],
                                        columns='EXPIRY_DT', aggfunc={'SETTLE_PR': np.sum,
                                                                      'OPEN_INT': np.sum})

    # calculate product-near and product-far
    data = pd.DataFrame([map(round, all_stocks['OPEN_INT'].iloc[:, 0] * all_stocks['SETTLE_PR'].iloc[:, 0])
                            , map(round, all_stocks['OPEN_INT'].iloc[:, 1] * all_stocks['SETTLE_PR'].iloc[:, 1] +
                                  all_stocks['OPEN_INT'].iloc[:, 2] * all_stocks['SETTLE_PR'].iloc[:, 2])]).T

    data.columns = ['Product-Near', 'Product-Far']
    data.index = all_stocks.index

    pd.set_option('display.precision', 7)
    return data




In [401]:
download_dir
# traverse through all bhavcopies

for r, d, f in os.walk(download_dir):
    for csv_file in f:
        file_name = csv_file
        data = bhavcopy_processor( file_name ) # Dataframe with product far and near
        print data


                                         Product-Near    Product-Far
SYMBOL     Sector        IsNifty Type                               
ACC        Cement        False   SSF    1.5414358e+09  3.9276984e+08
ADANIENT   Diversified   False   SSF    2.5759374e+09  4.4089466e+09
ADANIPORTS Infra         True    SSF    3.4493168e+09  1.3887344e+09
ADANIPOWER Power         False   SSF    1.7611620e+09  3.8040510e+09
AJANTPHARM Pharma        False   SSF    4.5837120e+08  1.5856035e+08
ALBK       PSUBANKS      False   SSF    3.8714830e+08  3.4579490e+08
AMARAJABAT chemicals     False   SSF    9.2505756e+08  1.6010746e+08
AMBUJACEM  Cement        False   SSF    2.1501032e+09  2.6032150e+08
APOLLOHOSP Pharma        False   SSF    9.7639820e+08  1.6897345e+08
APOLLOTYRE Auto          False   SSF    1.9018983e+09  3.7831710e+08
ARVIND     Consumer      False   SSF    1.8319210e+09  4.3257250e+08
ASHOKLEY   Auto          False   SSF    5.2956420e+09  1.2996810e+09
ASIANPAINT Consumer      True    S

In [533]:
expiry = 'E'

In [534]:
if expiry == 'E':
    pass
else :
    print 'calc'

In [536]:
import xlwings as xw
import numpy as np
import pandas as pd
from collections import OrderedDict
from time import time
from dateutil import parser
import os



master_dir = 'X:\\Data Analytics\\ETL\\Master\\'
download_dir = 'X:\\Data Analytics\\ETL\\Download\\'
output_dir = 'X:\\Data Analytics\\ETL\\Output\\'

# calculate SSF roll over

def ssf_calculator(a, b):
    try:
        return round(b / (a + b), 3)
    except:
        return 0.0

def bhavcopy_processor(file_name):
    '''Function to read and process bhavcopy
    Convert it into a pivot_table
    calculate product-far/ product-near and return that dataframe '''

    # read bhav copy as workbook
    wb = xw.Book(download_dir + file_name)

    # read sheet
    sht = wb.sheets[file_name[:-4]]

    # convert sheet to pandas dataframe
    all_stocks = sht.range('A1').options(pd.DataFrame, expand='table').value

    # select only FUTSTK and FUTIDX
    new_index = ['FUTSTK', 'FUTIDX']
    all_stocks = all_stocks.loc[new_index]

    # Categorization based on master document
    # read master doc
    mast_book = xw.Book(master_dir + 'MasterData.xlsx')

    # read sheet
    mast_sheet = mast_book.sheets('Sheet1')

    # convert sheet to pandas dataframe
    master = mast_sheet.range('A1').options(pd.DataFrame, expand='table').value
    # master.rename(columns={'SymbolName': 'SYMBOL'}, inplace=True)

    # filter master for symbol, bloomcode and sector
    master.set_index(keys='SYMBOL', inplace=True)
    master.drop(columns=['IsActiveFNO'], axis=1, inplace=True)

    # join dataframes master and all_stocks based on SYMBOL
    all_stocks = all_stocks.join(master, on='SYMBOL')

    # create Pivot table
    all_stocks = all_stocks.pivot_table(values=['OPEN_INT', 'SETTLE_PR'], index=['SYMBOL', 'Sector', 'IsNifty', 'Type'],
                                        columns='EXPIRY_DT', aggfunc={'SETTLE_PR': np.sum,
                                                                      'OPEN_INT': np.sum})

    # calculate product-near and product-far
    data = pd.DataFrame([map(round, all_stocks['OPEN_INT'].iloc[:, 0] * all_stocks['SETTLE_PR'].iloc[:, 0])
                            , map(round, all_stocks['OPEN_INT'].iloc[:, 1] * all_stocks['SETTLE_PR'].iloc[:, 1] +
                                  all_stocks['OPEN_INT'].iloc[:, 2] * all_stocks['SETTLE_PR'].iloc[:, 2])]).T

    data.columns = ['Product-Near', 'Product-Far']
    data.index = all_stocks.index

    # apply roll over to all stocks and append ssf_rollover
    data['SSF_rollover'] = data.apply(lambda row: ssf_calculator(row['Product-Near'], row['Product-Far']), axis=1)
    pd.set_option('display.precision', 10)
    return data, master


def sector_rollovers(data):
    '''Function to calculate sector vice rollovers and dump to base Report_stats.xlsx'''

    # Group all sectors and run sum() aggregate
    Sector = data.groupby('Sector').sum()
    Sector['Roll'] = Sector.apply(lambda row: ssf_calculator(row['Product-Near'], row['Product-Far']), axis=1)

    # append nifty stocks to Sector table
    nifty_product_near = data.groupby('IsNifty').get_group(True).sum()['Product-Near']
    nifty_product_far = data.groupby('IsNifty').get_group(True).sum()['Product-Far']
    nifty_roll = ssf_calculator(nifty_product_near, nifty_product_far)

    Nifty_Stocks_df = pd.DataFrame(
        {'Product-Near': [nifty_product_near], 'Product-Far': [nifty_product_far], 'Roll': [nifty_roll]},
        index=['Nifty Stocks'])

    # append non nifty stocks to sector table
    non_nifty_product_near = data.groupby('IsNifty').get_group(False).groupby('Type').sum()['Product-Near']['SSF']
    non_nifty_product_far = data.groupby('IsNifty').get_group(False).groupby('Type').sum()['Product-Far']['SSF']
    non_nifty_roll = ssf_calculator(non_nifty_product_near, non_nifty_product_far)

    Non_Nifty_Stocks_df = pd.DataFrame(
        {'Product-Near': [non_nifty_product_near], 'Product-Far': [non_nifty_product_far], 'Roll': [non_nifty_roll]},
        index=['Non-Nifty Stocks'])

    # append SSF stock to sector table
    ssf_df = pd.DataFrame({'Product-Near': data.groupby('Type').sum().loc['SSF']['Product-Near'],
                           'Product-Far': data.groupby('Type').sum().loc['SSF']['Product-Far'],
                           'Roll': ssf_calculator(data.groupby('Type').sum().loc['SSF']['Product-Near'],
                                                  data.groupby('Type').sum().loc['SSF']['Product-Far'])},
                          index=['SSF'])

    # append NIFTY to sector table
    NIFTY_df = pd.DataFrame({'Product-Near': data.loc['NIFTY'].loc[:, 'Product-Near'][0],
                             'Product-Far': data.loc['NIFTY'].loc[:, 'Product-Far'][0],
                             'Roll': ssf_calculator(data.loc['NIFTY'].loc[:, 'Product-Near'][0],
                                                    data.loc['NIFTY'].loc[:, 'Product-Far'][0])},
                            index=['Nifty'])

    # append BANKNIFTY to sector table
    BANKNIFTY_df = pd.DataFrame({'Product-Near': data.loc['BANKNIFTY'].loc[:, 'Product-Near'][0],
                                 'Product-Far': data.loc['BANKNIFTY'].loc[:, 'Product-Far'][0],
                                 'Roll': ssf_calculator(data.loc['BANKNIFTY'].loc[:, 'Product-Near'][0],
                                                        data.loc['BANKNIFTY'].loc[:, 'Product-Far'][0])},
                                index=['NiftyBank'])

    Overall = pd.DataFrame({'Product-Near': data['Product-Near'].sum(),
                            'Product-Far': data['Product-Far'].sum(),
                            'Roll': round(
                                data['Product-Far'].sum() / (data['Product-Near'].sum() + data['Product-Far'].sum()),
                                2)},
                           index=['Overall'])

    frames = [Sector, Nifty_Stocks_df, Non_Nifty_Stocks_df, ssf_df, NIFTY_df, BANKNIFTY_df, Overall]
    result = pd.concat(frames, sort=False)
    result.sort_index(inplace=True)
    result.drop(index=['None', 'Index'], axis=0, inplace=True)

    # write dataframe to excel
    writer = pd.ExcelWriter(output_dir+'Report_stats.xlsx', engine='xlsxwriter')
    result.to_excel(writer, sheet_name='SSF & Indices Report')
    return result




def rollover_data_dumper(expiry, month, year):
    '''Function to dump rollover data for expiry'''
    print 'Dumping data for {} {} {}.....'.format(expiry, month, year)

    # ignore index i.e consider only SSF stocks

    ssf_rollover = data.groupby('Type').get_group('SSF').reset_index(level=[1, 2, 3], drop=True)[['SSF_rollover']]

    # fetch bloomcode from master and join dataframes to dumper

    stock_bloom = master.groupby('Type').get_group('SSF')[['BloomCode']]
    dumper = stock_bloom.merge(ssf_rollover, on='SYMBOL')

    # dump expiry, month and year

    dumper['Expiry'], dumper['Month'], dumper['Year'] = expiry, month, year
    dumper.reset_index(inplace=True)

    # reaarange according to schema

    dumper = dumper[['Expiry', 'SYMBOL', 'BloomCode', 'Month', 'Year', 'SSF_rollover']]
    writer = pd.ExcelWriter(output_dir+'Report.xlsx', engine='xlsxwriter')
    dumper.to_excel(writer, sheet_name='SSF', index=False)

    # dump indices rollover

    indices_rollover = pd.DataFrame(result['Roll'])
    indices_rollover['Expiry'], indices_rollover['Month'], indices_rollover['Year'] = expiry, month, year
    indices_rollover.index.name = 'Indices'
    indices_rollover.reset_index(inplace=True)
    indices_rollover = indices_rollover[['Expiry', 'Month', 'Year', 'Indices', 'Roll']]
    indices_rollover.to_excel(writer, sheet_name='Indices', index=False)

    # Unrolled data

    unrolled = master.groupby('Type').get_group('SSF')[['BloomCode']].merge(
        data.groupby('Type').get_group('SSF').reset_index(
            level=[1, 2, 3], drop=True)[['Product-Near']], on='SYMBOL')

    # Convert product-near to million dollars rates

    dollar = pd.read_excel(master_dir+'USDINRRates.xlsx')
    dollar['Date'] = dollar.apply(lambda row: parser.parse(row['Date']), axis=1)
    dollar.set_index('Date', inplace=True)

    # extract date from bhavcopy filename and convert it to date object
    # traverse in dollar dump for that date and retrive its US dollar rate
    dollar_value = dollar.loc[file_name][0]

    unrolled['Million_dollar'] = unrolled['Product-Near'].apply(lambda x: round(x / (dollar_value * 1000000), 1))

    # dump million dollar rates of stocks

    unrolled_dumper = master.groupby('Type').get_group('SSF')[['BloomCode']]
    unrolled_dumper['Expiry'], unrolled_dumper['Month'], unrolled_dumper['Year'] = expiry, month, year
    unrolled_dumper['Million_dollar'] = unrolled['Million_dollar']
    unrolled_dumper.reset_index(inplace=True)
    unrolled_dumper = unrolled_dumper[['Expiry', 'SYMBOL', 'BloomCode', 'Month', 'Year', 'Million_dollar']]

    unrolled_dumper.to_excel(writer, sheet_name='Unrolled', index=False)



# Calculate avergae of last three months
def average_calculator(expiry, value, stock_names, data_df):
    '''Function to calculate average of last three months'''
    average = []
    for i in range(len(stock_names)):
        # average.append( round(rolldata.groupby('Expiry').get_group(expiry).loc[stock_names[i]][-3:]['SSF_rollover'].mean(), 2) )
        average.append(round(data_df.groupby('Expiry').get_group(expiry).loc[stock_names[i]][-3:][value].mean(), 2))
    return average


# Sort elements based on criteria top or low performing stocks
def sort_stocks(rank_order, sorting_type, data, criteria):
    sorted_stocks = {}

    for group_name, group_elements in data.groupby('TYPE'):
        sorted_stocks[group_name] = group_elements.sort_values(ascending=sorting_type, by=criteria)[: rank_order]

    return sorted_stocks


def greaterthanAverage(expiry ):
    '''Function for greater than average sheet'''

    # read from dumper object for SSF rolldata
    rolldata = pd.read_excel(master_dir + 'RollData.xlsx')
    rolldata = rolldata.set_index('SYMBOL')

    # average is appended in sorted manner
    stock_names = list(sorted(set(rolldata.index)))
    average_ssf = pd.DataFrame(average_calculator(expiry, 'SSF_rollover', stock_names, rolldata), index=stock_names,
                               columns=['Average'])
    average_ssf.index.name = 'SYMBOL'

    # Greater than average
    # read from dumped data
    grtthanaverage = pd.read_excel(output_dir + 'Report.xlsx', sheet_name='SSF')
    grtthanaverage = grtthanaverage.merge(average_ssf, on='SYMBOL')

    # difference of today's rollover and average of last 3 months
    grtthanaverage['Difference'] = grtthanaverage['SSF_rollover'] - grtthanaverage['Average']

    # read master for fetching type of stock nifty, non-nifty and physical
    type_of_stock = pd.read_excel(master_dir + 'MasterData.xlsx', sheet_name='Physical')
    grtthanaverage = grtthanaverage.merge(type_of_stock, on='SYMBOL')

    # pass False for top performing as sorting parameter
    grt_avg_top_stocks = sort_stocks(5, False, grtthanaverage, 'Difference')

    # pass True for low performing as sorting parameter
    grt_avg_low_stocks = sort_stocks(5, True, grtthanaverage, 'Difference')

    return grt_avg_top_stocks, grt_avg_low_stocks


def del_qty_average():
    '''Function to calculate last 3months del quantity average'''

    # read del qty dump
    del_qty = pd.read_excel(master_dir + 'Del Qty.xlsx')
    # parse del qty dates to date format
    del_qty['Date'] = del_qty.apply(lambda row: parser.parse(row['Date']), axis=1)
    del_qty.set_index('SYMBOL', inplace=True)
    stock_names = list(set(del_qty.index))
    average = {}
    for name in stock_names:
        average[name] = del_qty.loc[name].sort_values(ascending=False, by='Date').iloc[:3, -1].mean()

    average_del = pd.DataFrame(average.items(), columns=['SYMBOL', 'Del Avg'])
    average_del.dropna(inplace=True, axis=0)
    average_del.set_index('SYMBOL', inplace=True)
    return average_del


def unrolled_OI_Calculator(expiry):
    '''Function to compute unrolled OI sheet'''
    # unrolled OI calculator

    unrolled_today = pd.read_excel(master_dir + 'Unrolled_dumped.xlsx')
    unrolled_today = unrolled_today.set_index('SYMBOL')

    # average is appended in sorted manner
    stock_names = list(sorted(set(unrolled_today.index)))

    average_unrolled = pd.DataFrame(average_calculator(expiry, 'Million_dollar', stock_names, unrolled_today),
                                    index=stock_names, columns=['Average'])
    average_unrolled.index.name = 'SYMBOL'

    # read unrolled (million_dollar )from dumped data
    unrolled_OI_calc = pd.read_excel(output_dir + 'Report.xlsx', sheet_name='Unrolled')
    unrolled_OI_calc = unrolled_OI_calc.merge(average_unrolled, on='SYMBOL')

    # difference of today's million_unrolled and average of last 3 months
    unrolled_OI_calc['Difference'] = unrolled_OI_calc['Million_dollar'] - unrolled_OI_calc['Average']

    # Calculate percent for unrolled- avg/ avg
    unrolled_OI_calc['Percent'] = unrolled_OI_calc.apply(
        lambda row: round((row['Million_dollar'] - row['Average']) / row['Average'] * 100), axis=1)

    # fetch del qty averages for last three months
    del_qty = del_qty_average()
    unrolled_OI_calc = unrolled_OI_calc.merge(del_qty, on='SYMBOL')
    unrolled_OI_calc['Ratio'] = unrolled_OI_calc['Million_dollar'] / unrolled_OI_calc['Del Avg']
    unrolled_OI_calc['Ratio'] = unrolled_OI_calc['Ratio'].apply(lambda x: round(x))

    # read master for fetching type of stock nifty, non-nifty and physical
    type_of_stock = pd.read_excel(master_dir + 'MasterData.xlsx', sheet_name='Physical')
    unrolled_OI_calc = unrolled_OI_calc.merge(type_of_stock, on='SYMBOL')

    # pass False for top performing as sorting parameter (take only top stocks for unrolled)
    unrolled_OI_top_stocks = sort_stocks(5, False, unrolled_OI_calc, 'Ratio')
    return unrolled_OI_top_stocks


def increased_activity_calc(expiry, month, year):
    # increased Activity
    # read dumped value for SSF stucks for current expiry

    increased_activity = pd.read_excel(output_dir + 'Report.xlsx', sheet_name='SSF')
    increased_activity.set_index('SYMBOL', inplace=True)
    # read SSF stocks for T-1 expiry
    expiry_list = 'E-5 E-4 E-3 E-2 E-1 E'.split(' ')

    previous_day_expiry = pd.read_excel(master_dir + 'RollData.xlsx')
    previous_day_expiry = previous_day_expiry[
        (previous_day_expiry['Expiry'] == expiry_list[expiry_list.index(expiry) - 1]) & (
                previous_day_expiry['Month'] == month) &
        (previous_day_expiry['Year'] == int(year))][['SYMBOL', 'SSF_rollover']]
    previous_day_expiry.set_index('SYMBOL', inplace=True)

    # merge on symbol

    increased_activity = increased_activity.merge(previous_day_expiry, on='SYMBOL')
    increased_activity.columns = ['Expiry', 'BloomCode', 'Month', 'Year', 'SSF_rollover_T', 'SSF_rollover_T-1']

    # calculate difference Today - (T-1)

    increased_activity['Difference'] = increased_activity['SSF_rollover_T'] - increased_activity['SSF_rollover_T-1']
    increased_activity = increased_activity.sort_values(ascending=False, by='Difference').head(10)
    return increased_activity


def leading_lagging_sectors(expiry):
    '''Function to fetch leading and lagging sectors'''

    # read dumped indices
    dumped_indices = pd.read_excel(master_dir + 'Indices_dump.xlsx')
    dumped_indices = dumped_indices.groupby('Expiry').get_group(expiry)
    indices_names = list(sorted(set(dumped_indices['Indices'])))
    dumped_indices.set_index('Indices', inplace=True)

    # Average of indicies for last 3 months
    avg_indices = pd.DataFrame(average_calculator(expiry, 'Roll', indices_names, dumped_indices),
                               index=indices_names, columns=['Average'])
    avg_indices.index.name = 'Indices'

    indices_read = pd.read_excel(output_dir + 'Report.xlsx', sheet_name='Indices')
    indices_diff = indices_read.set_index('Indices')
    indices_diff = indices_diff.merge(avg_indices, on='Indices')
    indices_diff['Difference'] = indices_diff['Roll'] - indices_diff['Average']
    # Sectors to ignore
    exclude_list = ['Nifty', 'NiftyBank', 'Nifty Stocks', 'Non-Nifty Stocks', 'SSF', 'Overall', 'Real Estate',
                    'Diversified', 'chemicals', 'Media']
    new_indices = indices_diff.drop(index=exclude_list)
    lagging_sectors = str(new_indices.sort_values('Difference')[:2].index[0]), str(
        new_indices.sort_values('Difference')[:2].index[1])
    leading_sectors = str(new_indices.sort_values('Difference', ascending=False)[:2].index[0]), str(
        new_indices.sort_values('Difference')[:2].index[1])

    return leading_sectors, lagging_sectors, avg_indices


def print_final_report(expiry, avg_indices, increased_activity, leading_sectors, lagging_sectors,
                       grt_avg_top_stocks, grt_avg_low_stocks, unrolled_OI_top_stocks):
    '''Function to print final report to excel'''

    print_data = []

    print_data.append('----- Highlights at {0} -----'.format(expiry))

    for name in ['Nifty', 'NiftyBank', 'SSF', 'Nifty Stocks', 'Overall']:
        indices_read = pd.read_excel(output_dir + 'Report.xlsx', sheet_name='Indices')
        value = indices_read[indices_read['Indices'] == name]['Roll'].values[0]

        avg = avg_indices.loc[name].values[0]
        diff = value - avg
        if diff > 0:
            print_data.append(
                ' *{0} rolls ({1}%) are {2} percentage points higher than the 3 month average of {3}%. '.format(
                    name, value * 100, diff * 100, avg * 100))
        else:
            print_data.append(
                ' *{0} rolls ({1}%) are {2} percentage points lower than the 3 month average of {3}%. '.format(
                    name, value * 100, diff * -100, avg * 100))

    if expiry in ['E-5', 'E-4', 'E-3']:
        pass
    else:
        print_data.append(' *Increased roll activity was seen in {0} '.format(
            ', '.join(str(x) for x in increased_activity['BloomCode'].values)))

    print_data.append(
        '*Rolls in {0} and {1} names are leading their 3 month average, while those in {2} and {3} are lagging'.format(
            leading_sectors[0],
            leading_sectors[1],
            lagging_sectors[0],
            lagging_sectors[1]))
    print_data.append(' *Stocks with rollovers higher than their 3 month average: ')
    i = 1

    data = OrderedDict(reversed(list(grt_avg_top_stocks.items())))
    while True:
        for key, values in data.iteritems():
            print_data.append(' {0} '.format(key))
            stocks = []
            for name in values['BloomCode'].values:
                stocks.append("{0}({1}%)".format(str(name),
                                                 int(round(
                                                     values.set_index('BloomCode').loc[name][['SSF_rollover']][
                                                         0] * 100))))
            print_data.append(', '.join(stocks))
        i += 1
        if i > 2:
            break
        data = OrderedDict(reversed(list(grt_avg_low_stocks.items())))
        print_data.append(' *Stocks with rollovers lower than their 3 month average: ')

    print_data.append(
        ' *SSF with high unrolled OI compared to 3 mn avg: (OI in $mn and Ratio of OI to 3mn avg of expiry day delivery )')

    for key, values in OrderedDict(reversed(list(unrolled_OI_top_stocks.items()))).iteritems():
        print_data.append(' {0} '.format(key))
        stocks = []
        for name in values['BloomCode']:
            stocks.append(
                '{0}({1}/{2}x)'.format(name, values.set_index('BloomCode').loc[name]['Million_dollar'],
                                       values.set_index('BloomCode').loc[name]['Ratio']))
        print_data.append(', '.join(stocks))

    pd.DataFrame(print_data).to_excel(output_dir+'Final.xlsx', sheet_name='Report', header=None, index=None)






start_time = time()

# traverse through all bhavcopies
for r, d, f in os.walk(download_dir):
    for csv_file in f:
        file_name = csv_file
        data, master = bhavcopy_processor( file_name ) # Dataframe with product far and near and rollover values
        result = sector_rollovers(data)

        # extract date from input bhavcopy name
        file_name = file_name[2:11]
        date, month, year = file_name[:2], file_name[2:5], file_name[5:]

        # read expiry from master dump to get expiry code
        dates = pd.read_excel(master_dir+'Expiry_dates_master.xlsx')
        expiry = str(dates[(dates['Date'] == int(date))
                           & (dates['Month'] == month) & (dates['Year'] == int(year))]['Expiry'].values[0])

        rollover_data_dumper(expiry, month, year)

        # get top and low stocks from grt than avg sheet
        grt_avg_top_stocks, grt_avg_low_stocks = greaterthanAverage(expiry)
        # get top unrolled OI stocks
        if expiry == 'E':
            pass
        else:
            # calculate unrolled OI
            unrolled_OI_top_stocks = unrolled_OI_Calculator(expiry)
        # increased activity
        if expiry in ['E-5','E-4','E-3']:
            pass
        else:
            #calculate increased activity
            increased_activity = increased_activity_calc(expiry, month, year)

        # get top 2 leading and lagging sectors
        leading_sectors, lagging_sectors, avg_indices  = leading_lagging_sectors(expiry)

        # call final print
        print_final_report(expiry, avg_indices, increased_activity, leading_sectors, lagging_sectors,
                           grt_avg_top_stocks, grt_avg_low_stocks, unrolled_OI_top_stocks)



end_time = time()


print "Execution time: ", end_time - start_time

C:\ProgramData\Anaconda2\lib\site-packages\ipykernel_launcher.py:19: RuntimeWarning: invalid value encountered in double_scalars


Dumping data for E-3 OCT 2018.....
Execution time:  7.63900017738


In [217]:
def average_calculator(expiry, value, stock_names, data_df, month):
    '''Function to calculate average of last three months'''
    months = {"JAN":0, "FEB":1, "MAR":2, "APR":3, "MAY":4, "JUN":5, "JUL":6, "AUG":7, "SEP":8, "OCT":9, "NOV":10, "DEC":11 }
    #months.keys()[months.values().index(months[month]-1)]
    average = []


    for i in range(len(stock_names)):
        # average.append( round(rolldata.groupby('Expiry').get_group(expiry).loc[stock_names[i]][-3:]['SSF_rollover'].mean(), 2) )
        average.append(round(data_df[(data_df['Expiry']==expiry) &
         ((data_df['Month']==months.keys()[months.values().index(months[month]-1)]) |
        (data_df['Month']==months.keys()[months.values().index(months[month]-2)]) |
        (data_df['Month']==months.keys()[months.values().index(months[month]-3)])) ].loc[stock_names[i]][value].mean(), 1))

    return average

In [226]:
# read dumped indices
dumped_indices = pd.read_excel(master_dir + 'Indices_dump.xlsx')
dumped_indices = dumped_indices.groupby('Expiry').get_group(expiry)
indices_names = list(sorted(set(dumped_indices['Indices'])))
dumped_indices.set_index('Indices', inplace=True)

# Average of indicies for last 3 months
avg_indices = pd.DataFrame(average_calculator(expiry, 'Roll', indices_names, dumped_indices, month),
                           index=indices_names, columns=['Average'])
avg_indices.index.name = 'Indices'


indices_read = pd.read_excel(output_dir + 'Report.xlsx', sheet_name='Indices')
indices_diff = indices_read.set_index('Indices')
indices_diff = indices_diff.merge(avg_indices, on='Indices')
indices_diff['Difference'] = indices_diff['Roll'] - indices_diff['Average']
# Sectors to ignore
exclude_list = ['Nifty', 'NiftyBank', 'Nifty Stocks', 'Non-Nifty Stocks', 'SSF', 'Overall', 'Real Estate',
                'Diversified', 'chemicals', 'Media']
new_indices = indices_diff.drop(index=exclude_list)
lagging_sectors = str(new_indices.sort_values('Difference')[:2].index[0]), str(
    new_indices.sort_values('Difference')[:2].index[1])
leading_sectors = str(new_indices.sort_values('Difference', ascending=False)[:2].index[0]), str(
    new_indices.sort_values('Difference', ascending=False)[:2].index[1])

# check for zero leading and lagging sectors
print leading_sectors, lagging_sectors



('Power', 'Private Banks') ('Pharma', 'Telecom')


In [231]:
new_indices.sort_values('Difference')[:2]

Expiry Month  Year   Roll  Average  Difference
Indices                                               
Pharma     E-3   OCT  2018  0.179      0.3      -0.121
Telecom    E-3   OCT  2018  0.307      0.4      -0.093

In [245]:
new_indices.sort_values('Difference')[:2].index[1]

u'Telecom'

In [244]:
new_indices.sort_values('Difference')[:2].iloc[-1:,-1][0]

-0.09300000000000003

In [ ]:
if new_indices.sort_values('Difference')[:2].iloc[:-1,-1][0] != 0.0:
    lagging_sectors.append(new_indices.sort_values('Difference')[:2].index[0])
if new_indices.sort_values('Difference')[:2].iloc[-1:,-1][0] != 0.0:
    lagging_sectors.append(new_indices.sort_values('Difference')[:2].index[1])

    

In [246]:
new_indices.sort_values('Difference')[:2].iloc[:-1,-1][0]

-0.121

In [255]:
new_indices.sort_values('Difference', ascending = False)[:2]

Expiry Month  Year   Roll  Average  Difference
Indices                                                     
Power            E-3   OCT  2018  0.337      0.2       0.137
Private Banks    E-3   OCT  2018  0.286      0.2       0.086

In [247]:
new_indices.sort_values('Difference', ascending = False)[:2].iloc[:-1,-1][0]

0.137

In [260]:
new_indices.sort_values('Difference', ascending = False)[:2].iloc[-1:,-1][0]

0.08599999999999997

In [262]:
new_indices.sort_values('Difference')[:2]

Expiry Month  Year   Roll  Average  Difference
Indices                                               
Pharma     E-3   OCT  2018  0.179      0.3      -0.121
Telecom    E-3   OCT  2018  0.307      0.4      -0.093

In [274]:
new_indices.sort_values('Difference')[:2].index[0]

u'Pharma'

In [267]:
new_indices.sort_values('Difference', ascending = False)[:2].iloc[:-1,-1][0]

0.137

In [278]:
def del_qty_average():
    '''Function to calculate last 3months del quantity average'''

    # read del qty dump
    del_qty = pd.read_excel(master_dir + 'Del Qty.xlsx')
    # parse del qty dates to date format
    del_qty['Date'] = del_qty.apply(lambda row: parser.parse(row['Date']), axis=1)
    del_qty.set_index('SYMBOL', inplace=True)
    stock_names = list(set(del_qty.index))
    average = {}
    for name in stock_names:
        average[name] = round(del_qty.loc[name].sort_values(ascending=False, by='Date').iloc[:3, -1].mean(),2)

    average_del = pd.DataFrame(average.items(), columns=['SYMBOL', 'Del Avg'])
    average_del.dropna(inplace=True, axis=0)
    average_del.set_index('SYMBOL', inplace=True)
    return average_del



def sort_stocks(rank_order, sorting_type, data, criteria):
    sorted_stocks = {}

    for group_name, group_elements in data.groupby('TYPE'):
        sorted_stocks[group_name] = group_elements.sort_values(ascending=sorting_type, by=criteria)[: rank_order]

    return sorted_stocks

In [279]:
'''Function to compute unrolled OI sheet'''
# unrolled OI calculator

unrolled_today = pd.read_excel(master_dir + 'Unrolled_dumped.xlsx')
unrolled_today = unrolled_today.set_index('SYMBOL')

# average is appended in sorted manner
stock_names = list(sorted(set(unrolled_today.index)))

average_unrolled = pd.DataFrame(average_calculator(expiry, 'Million_dollar', stock_names, unrolled_today, month),
                                index=stock_names, columns=['Average'])
average_unrolled.index.name = 'SYMBOL'

# read unrolled (million_dollar )from dumped data
unrolled_OI_calc = pd.read_excel(output_dir + 'Report.xlsx', sheet_name='Unrolled')
unrolled_OI_calc = unrolled_OI_calc.merge(average_unrolled, on='SYMBOL')

# difference of today's million_unrolled and average of last 3 months
unrolled_OI_calc['Difference'] = unrolled_OI_calc['Million_dollar'] - unrolled_OI_calc['Average']

# Calculate percent for unrolled- avg/ avg
unrolled_OI_calc['Percent'] = unrolled_OI_calc.apply(
    lambda row: round((row['Million_dollar'] - row['Average']) / row['Average'] * 100), axis=1)

# fetch del qty averages for last three months
del_qty = del_qty_average()
unrolled_OI_calc = unrolled_OI_calc.merge(del_qty, on='SYMBOL')
unrolled_OI_calc['Ratio'] = unrolled_OI_calc['Million_dollar'] / unrolled_OI_calc['Del Avg']
unrolled_OI_calc['Ratio'] = unrolled_OI_calc['Ratio'].apply(lambda x: round(x))

# read master for fetching type of stock nifty, non-nifty and physical
type_of_stock = pd.read_excel(master_dir + 'MasterData.xlsx', sheet_name='Physical')
unrolled_OI_calc = unrolled_OI_calc.merge(type_of_stock, on='SYMBOL')

# pass False for top performing as sorting parameter (take only top stocks for unrolled)
unrolled_OI_top_stocks = sort_stocks(5, False, unrolled_OI_calc, 'Ratio')
print unrolled_OI_top_stocks
  


{u'Physical':     Expiry   SYMBOL BloomCode Month  Year  Million_dollar  Average  \
76     E-3     IFCI      IFCI   OCT  2018             8.7     15.5   
192    E-3     MRPL      MRPL   OCT  2018             4.9      7.9   
72     E-3     IDBI      IDBI   OCT  2018            27.7     51.3   
40     E-3  CGPOWER   CGPOWER   OCT  2018            14.0     21.6   
186    E-3     GSFC      GSFC   OCT  2018            10.0     26.3   

     Difference  Percent  Del Avg  Ratio      TYPE  
76         -6.8    -44.0     0.34   26.0  Physical  
192        -3.0    -38.0     0.21   23.0  Physical  
72        -23.6    -46.0     1.26   22.0  Physical  
40         -7.6    -35.0     0.69   20.0  Physical  
186       -16.3    -62.0     0.63   16.0  Physical  , u'Non Nifty':     Expiry      SYMBOL BloomCode Month  Year  Million_dollar  Average  \
34     E-3  CENTURYTEX      CENT   OCT  2018            47.6     57.8   
147    E-3   TATAPOWER      TPWR   OCT  2018            36.3     50.6   
95     E-3   

In [307]:
import xlwings as xw
import numpy as np
import pandas as pd
from collections import OrderedDict
from time import time
from dateutil import parser
import os

master_dir = 'X:\\Data_Analytics\\ETL\\Master\\'
download_dir = 'X:\\Data_Analytics\\ETL\\Download\\'
output_dir = 'X:\\Data_Analytics\\ETL\\Output\\'


# calculate SSF roll over

def ssf_calculator(a, b):
    try:
        return round(b / (a + b), 3)
    except:
        return 0.0


def bhavcopy_processor(file_name):
    '''Function to read and process bhavcopy
    Convert it into a pivot_table
    calculate product-far/ product-near and return that dataframe '''

    # read bhav copy as workbook
    wb = xw.Book(download_dir + file_name)

    # read sheet
    sht = wb.sheets[file_name[:-4]]

    # convert sheet to pandas dataframe
    all_stocks = sht.range('A1').options(pd.DataFrame, expand='table').value

    # select only FUTSTK and FUTIDX
    new_index = ['FUTSTK', 'FUTIDX']
    all_stocks = all_stocks.loc[new_index]

    # Categorization based on master document
    # read master doc
    mast_book = xw.Book(master_dir + 'MasterData.xlsx')

    # read sheet
    mast_sheet = mast_book.sheets('Sheet1')

    # convert sheet to pandas dataframe
    master = mast_sheet.range('A1').options(pd.DataFrame, expand='table').value
    # master.rename(columns={'SymbolName': 'SYMBOL'}, inplace=True)

    # filter master for symbol, bloomcode and sector
    master.set_index(keys='SYMBOL', inplace=True)
    master.drop(columns=['IsActiveFNO'], axis=1, inplace=True)

    # join dataframes master and all_stocks based on SYMBOL
    all_stocks = all_stocks.join(master, on='SYMBOL')

    # create Pivot table
    all_stocks = all_stocks.pivot_table(values=['OPEN_INT', 'SETTLE_PR'], index=['SYMBOL', 'Sector', 'IsNifty', 'Type'],
                                        columns='EXPIRY_DT', aggfunc={'SETTLE_PR': np.sum,
                                                                      'OPEN_INT': np.sum})

    # calculate product-near and product-far
    data = pd.DataFrame([map(round, all_stocks['OPEN_INT'].iloc[:, 0] * all_stocks['SETTLE_PR'].iloc[:, 0])
                            , map(round, all_stocks['OPEN_INT'].iloc[:, 1] * all_stocks['SETTLE_PR'].iloc[:, 1] +
                                  all_stocks['OPEN_INT'].iloc[:, 2] * all_stocks['SETTLE_PR'].iloc[:, 2])]).T

    data.columns = ['Product-Near', 'Product-Far']
    data.index = all_stocks.index

    # apply roll over to all stocks and append ssf_rollover
    data['SSF_rollover'] = data.apply(lambda row: ssf_calculator(row['Product-Near'], row['Product-Far']), axis=1)
    pd.set_option('display.precision', 10)
    return data, master


def sector_rollovers(data):
    '''Function to calculate sector vice rollovers and dump to base Report_stats.xlsx'''

    # Group all sectors and run sum() aggregate
    Sector = data.groupby('Sector').sum()
    Sector['Roll'] = Sector.apply(lambda row: ssf_calculator(row['Product-Near'], row['Product-Far']), axis=1)
    Sector.drop(columns='SSF_rollover', inplace=True, axis=1)

    # append nifty stocks to Sector table
    nifty_product_near = data.groupby('IsNifty').get_group(True).sum()['Product-Near']
    nifty_product_far = data.groupby('IsNifty').get_group(True).sum()['Product-Far']
    nifty_roll = ssf_calculator(nifty_product_near, nifty_product_far)

    Nifty_Stocks_df = pd.DataFrame(
        {'Product-Near': [nifty_product_near], 'Product-Far': [nifty_product_far], 'Roll': [nifty_roll]},
        index=['Nifty Stocks'])

    # append non nifty stocks to sector table
    non_nifty_product_near = data.groupby('IsNifty').get_group(False).groupby('Type').sum()['Product-Near']['SSF']
    non_nifty_product_far = data.groupby('IsNifty').get_group(False).groupby('Type').sum()['Product-Far']['SSF']
    non_nifty_roll = ssf_calculator(non_nifty_product_near, non_nifty_product_far)

    Non_Nifty_Stocks_df = pd.DataFrame(
        {'Product-Near': [non_nifty_product_near], 'Product-Far': [non_nifty_product_far], 'Roll': [non_nifty_roll]},
        index=['Non-Nifty Stocks'])

    # append SSF stock to sector table
    ssf_df = pd.DataFrame({'Product-Near': data.groupby('Type').sum().loc['SSF']['Product-Near'],
                           'Product-Far': data.groupby('Type').sum().loc['SSF']['Product-Far'],
                           'Roll': ssf_calculator(data.groupby('Type').sum().loc['SSF']['Product-Near'],
                                                  data.groupby('Type').sum().loc['SSF']['Product-Far'])},
                          index=['SSF'])

    # append NIFTY to sector table
    NIFTY_df = pd.DataFrame({'Product-Near': data.loc['NIFTY'].loc[:, 'Product-Near'][0],
                             'Product-Far': data.loc['NIFTY'].loc[:, 'Product-Far'][0],
                             'Roll': ssf_calculator(data.loc['NIFTY'].loc[:, 'Product-Near'][0],
                                                    data.loc['NIFTY'].loc[:, 'Product-Far'][0])},
                            index=['Nifty'])

    # append BANKNIFTY to sector table
    BANKNIFTY_df = pd.DataFrame({'Product-Near': data.loc['BANKNIFTY'].loc[:, 'Product-Near'][0],
                                 'Product-Far': data.loc['BANKNIFTY'].loc[:, 'Product-Far'][0],
                                 'Roll': ssf_calculator(data.loc['BANKNIFTY'].loc[:, 'Product-Near'][0],
                                                        data.loc['BANKNIFTY'].loc[:, 'Product-Far'][0])},
                                index=['NiftyBank'])

    Overall = pd.DataFrame({'Product-Near': data['Product-Near'].sum(),
                            'Product-Far': data['Product-Far'].sum(),
                            'Roll': round(
                                data['Product-Far'].sum() / (data['Product-Near'].sum() + data['Product-Far'].sum()),
                                2)},
                           index=['Overall'])

    frames = [Sector, Nifty_Stocks_df, Non_Nifty_Stocks_df, ssf_df, NIFTY_df, BANKNIFTY_df, Overall]
    result = pd.concat(frames, sort=False)
    result.sort_index(inplace=True)
    result.drop(index=['None', 'Index'], axis=0, inplace=True)

    # write dataframe to excel
    writer = pd.ExcelWriter(output_dir + 'Report_stats.xlsx', engine='xlsxwriter')
    result.to_excel(writer, sheet_name='SSF & Indices Report')
    return result


def rollover_data_dumper(data, master, result, expiry, month, year, file_name):
    '''Function to dump rollover data for expiry'''
    print 'Dumping data for {} {} {}.....'.format(expiry, month, year)

    # ignore index i.e consider only SSF stocks

    ssf_rollover = data.groupby('Type').get_group('SSF').reset_index(level=[1, 2, 3], drop=True)[['SSF_rollover']]

    # fetch bloomcode from master and join dataframes to dumper

    stock_bloom = master.groupby('Type').get_group('SSF')[['BloomCode']]
    dumper = stock_bloom.merge(ssf_rollover, on='SYMBOL')

    # dump expiry, month and year

    dumper['Expiry'], dumper['Month'], dumper['Year'] = expiry, month, year
    dumper.reset_index(inplace=True)

    # reaarange according to schema

    dumper = dumper[['Expiry', 'SYMBOL', 'BloomCode', 'Month', 'Year', 'SSF_rollover']]
    writer = pd.ExcelWriter(output_dir + 'Report.xlsx', engine='xlsxwriter')
    dumper.to_excel(writer, sheet_name='SSF', index=False)

    # dump indices rollover

    indices_rollover = pd.DataFrame(result['Roll'])
    indices_rollover['Expiry'], indices_rollover['Month'], indices_rollover['Year'] = expiry, month, year
    indices_rollover.index.name = 'Indices'
    indices_rollover.reset_index(inplace=True)
    indices_rollover = indices_rollover[['Expiry', 'Month', 'Year', 'Indices', 'Roll']]
    indices_rollover.to_excel(writer, sheet_name='Indices', index=False)

    # Unrolled data

    unrolled = master.groupby('Type').get_group('SSF')[['BloomCode']].merge(
        data.groupby('Type').get_group('SSF').reset_index(
            level=[1, 2, 3], drop=True)[['Product-Near']], on='SYMBOL')

    # Convert product-near to million dollars rates

    dollar = pd.read_excel(master_dir + 'USDINRRates.xlsx')
    dollar['Date'] = dollar.apply(lambda row: parser.parse(row['Date']), axis=1)
    dollar.set_index('Date', inplace=True)

    # extract date from bhavcopy filename and convert it to date object
    # traverse in dollar dump for that date and retrive its US dollar rate

    dollar_value = dollar.loc[file_name]['Rate'][0]

    unrolled['Million_dollar'] = unrolled['Product-Near'].apply(lambda x: round(x / (dollar_value * 1000000), 1))

    # dump million dollar rates of stocks

    unrolled_dumper = master.groupby('Type').get_group('SSF')[['BloomCode']]
    unrolled_dumper['Expiry'], unrolled_dumper['Month'], unrolled_dumper['Year'] = expiry, month, year
    unrolled_dumper['Million_dollar'] = unrolled['Million_dollar']
    unrolled_dumper.reset_index(inplace=True)
    unrolled_dumper = unrolled_dumper[['Expiry', 'SYMBOL', 'BloomCode', 'Month', 'Year', 'Million_dollar']]

    unrolled_dumper.to_excel(writer, sheet_name='Unrolled', index=False)

    return dumper, indices_rollover, unrolled_dumper


# Calculate avergae of last three months
def average_calculator(expiry, value, stock_names, data_df, month):
    '''Function to calculate average of last three months'''
    months = {"JAN":0, "FEB":1, "MAR":2, "APR":3, "MAY":4, "JUN":5, "JUL":6, "AUG":7, "SEP":8, "OCT":9, "NOV":10, "DEC":11 }
    #months.keys()[months.values().index(months[month]-1)]
    average = []


    for i in range(len(stock_names)):
        # average.append( round(rolldata.groupby('Expiry').get_group(expiry).loc[stock_names[i]][-3:]['SSF_rollover'].mean(), 2) )
        average.append(round(data_df[(data_df['Expiry']==expiry) &
         ((data_df['Month']==months.keys()[months.values().index(months[month]-1)]) |
        (data_df['Month']==months.keys()[months.values().index(months[month]-2)]) |
        (data_df['Month']==months.keys()[months.values().index(months[month]-3)])) ].loc[stock_names[i]][value].mean(), 2))

    return average


# Sort elements based on criteria top or low performing stocks
def sort_stocks(rank_order, sorting_type, data, criteria):
    sorted_stocks = {}

    for group_name, group_elements in data.groupby('TYPE'):
        sorted_stocks[group_name] = group_elements.sort_values(ascending=sorting_type, by=criteria)[: rank_order]

    return sorted_stocks


def greaterthanAverage(expiry, month):
    '''Function for greater than average sheet'''

    # read from dumper object for SSF rolldata
    rolldata = pd.read_excel(master_dir + 'RollData.xlsx')
    rolldata = rolldata.set_index('SYMBOL')

    # average is appended in sorted manner
    stock_names = list(sorted(set(rolldata.index)))
    average_ssf = pd.DataFrame(average_calculator(expiry, 'SSF_rollover', stock_names, rolldata, month), index=stock_names,
                               columns=['Average'])
    average_ssf.index.name = 'SYMBOL'


    # Greater than average
    # read from dumped data
    grtthanaverage = pd.read_excel(output_dir + 'Report.xlsx', sheet_name='SSF')

    grtthanaverage = grtthanaverage.merge(average_ssf, on='SYMBOL')

    # difference of today's rollover and average of last 3 months
    grtthanaverage['Difference'] = grtthanaverage['SSF_rollover'] - grtthanaverage['Average']

    # read master for fetching type of stock nifty, non-nifty and physical
    type_of_stock = pd.read_excel(master_dir + 'MasterData.xlsx', sheet_name='Physical')
    grtthanaverage = grtthanaverage.merge(type_of_stock, on='SYMBOL')

    # pass False for top performing as sorting parameter
    grt_avg_top_stocks = sort_stocks(5, False, grtthanaverage, 'Difference')

    # pass True for low performing as sorting parameter
    grt_avg_low_stocks = sort_stocks(5, True, grtthanaverage, 'Difference')

    return grt_avg_top_stocks, grt_avg_low_stocks


def del_qty_average():
    '''Function to calculate last 3months del quantity average'''

    # read del qty dump
    del_qty = pd.read_excel(master_dir + 'Del Qty.xlsx')
    # parse del qty dates to date format
    del_qty['Date'] = del_qty.apply(lambda row: parser.parse(row['Date']), axis=1)
    del_qty.set_index('SYMBOL', inplace=True)
    stock_names = list(set(del_qty.index))
    average = {}
    for name in stock_names:
        average[name] = round(del_qty.loc[name].sort_values(ascending=False, by='Date').iloc[:3, -1].mean(),2)

    average_del = pd.DataFrame(average.items(), columns=['SYMBOL', 'Del Avg'])
    average_del.dropna(inplace=True, axis=0)
    average_del.set_index('SYMBOL', inplace=True)
    return average_del


def unrolled_OI_Calculator(expiry, month):
    '''Function to compute unrolled OI sheet'''
    # unrolled OI calculator

    unrolled_today = pd.read_excel(master_dir + 'Unrolled_dumped.xlsx')
    unrolled_today = unrolled_today.set_index('SYMBOL')

    # average is appended in sorted manner
    stock_names = list(sorted(set(unrolled_today.index)))

    average_unrolled = pd.DataFrame(average_calculator(expiry, 'Million_dollar', stock_names, unrolled_today, month),
                                    index=stock_names, columns=['Average'])
    average_unrolled.index.name = 'SYMBOL'

    # read unrolled (million_dollar )from dumped data
    unrolled_OI_calc = pd.read_excel(output_dir + 'Report.xlsx', sheet_name='Unrolled')
    unrolled_OI_calc = unrolled_OI_calc.merge(average_unrolled, on='SYMBOL')

    # difference of today's million_unrolled and average of last 3 months
    unrolled_OI_calc['Difference'] = unrolled_OI_calc['Million_dollar'] - unrolled_OI_calc['Average']

    # Calculate percent for unrolled- avg/ avg
    unrolled_OI_calc['Percent'] = unrolled_OI_calc.apply(
        lambda row: round((row['Million_dollar'] - row['Average']) / row['Average'] * 100), axis=1)

    # fetch del qty averages for last three months
    del_qty = del_qty_average()
    unrolled_OI_calc = unrolled_OI_calc.merge(del_qty, on='SYMBOL')
    unrolled_OI_calc['Ratio'] = unrolled_OI_calc['Million_dollar'] / unrolled_OI_calc['Del Avg']
    unrolled_OI_calc['Ratio'] = unrolled_OI_calc['Ratio'].apply(lambda x: round(x))

    # read master for fetching type of stock nifty, non-nifty and physical
    type_of_stock = pd.read_excel(master_dir + 'MasterData.xlsx', sheet_name='Physical')
    unrolled_OI_calc = unrolled_OI_calc.merge(type_of_stock, on='SYMBOL')

    # pass False for top performing as sorting parameter (take only top stocks for unrolled)
    unrolled_OI_top_stocks = sort_stocks(5, False, unrolled_OI_calc, 'Percent')
    return unrolled_OI_top_stocks


def increased_activity_calc(expiry, month, year):
    # increased Activity
    # read dumped value for SSF stucks for current expiry

    increased_activity = pd.read_excel(output_dir + 'Report.xlsx', sheet_name='SSF')
    increased_activity.set_index('SYMBOL', inplace=True)
    # read SSF stocks for T-1 expiry
    expiry_list = 'E-5 E-4 E-3 E-2 E-1 E'.split(' ')

    previous_day_expiry = pd.read_excel(master_dir + 'RollData.xlsx')
    previous_day_expiry = previous_day_expiry[
        (previous_day_expiry['Expiry'] == expiry_list[expiry_list.index(expiry) - 1]) & (
                previous_day_expiry['Month'] == month) &
        (previous_day_expiry['Year'] == int(year))][['SYMBOL', 'SSF_rollover']]
    previous_day_expiry.set_index('SYMBOL', inplace=True)

    # merge on symbol

    increased_activity = increased_activity.merge(previous_day_expiry, on='SYMBOL')
    increased_activity.columns = ['Expiry', 'BloomCode', 'Month', 'Year', 'SSF_rollover_T', 'SSF_rollover_T-1']

    # calculate difference Today - (T-1)

    increased_activity['Difference'] = increased_activity['SSF_rollover_T'] - increased_activity['SSF_rollover_T-1']
    increased_activity = increased_activity.sort_values(ascending=False, by='Difference').head(10)
    return increased_activity


def leading_lagging_sectors(expiry, month):
    '''Function to fetch leading and lagging sectors'''

    # read dumped indices
    dumped_indices = pd.read_excel(master_dir + 'Indices_dump.xlsx')
    dumped_indices = dumped_indices.groupby('Expiry').get_group(expiry)
    indices_names = list(sorted(set(dumped_indices['Indices'])))
    dumped_indices.set_index('Indices', inplace=True)

    # Average of indicies for last 3 months
    avg_indices = pd.DataFrame(average_calculator(expiry, 'Roll', indices_names, dumped_indices, month),
                               index=indices_names, columns=['Average'])
    avg_indices.index.name = 'Indices'



    indices_read = pd.read_excel(output_dir + 'Report.xlsx', sheet_name='Indices')
    indices_diff = indices_read.set_index('Indices')
    indices_diff = indices_diff.merge(avg_indices, on='Indices')
    indices_diff['Difference'] = indices_diff['Roll'] - indices_diff['Average']
    indices_diff['Difference'] = indices_diff.apply(lambda row: round(row['Difference'],2), axis=1)
    # Sectors to ignore
    exclude_list = ['Nifty', 'NiftyBank', 'Nifty Stocks', 'Non-Nifty Stocks', 'SSF', 'Overall', 'Real Estate',
                    'Diversified', 'chemicals', 'Media']
    new_indices = indices_diff.drop(index=exclude_list)
    leading_sectors = []
    lagging_sectors = []
    # append leading and lagging sectors
    if new_indices.sort_values('Difference', ascending = False)[:2].iloc[:-1,-1][0] == 0.0:
        pass
    else:
        leading_sectors.append(new_indices.sort_values('Difference', ascending = False)[:2].index[0])

    if new_indices.sort_values('Difference', ascending = False)[:2].iloc[-1:, -1][0] == 0.0:
        pass
    else:
        leading_sectors.append(new_indices.sort_values('Difference', ascending = False)[:2].index[1])

    if new_indices.sort_values('Difference')[:2].iloc[:-1,-1][0] == 0.0:
        pass
    else:
        lagging_sectors.append(new_indices.sort_values('Difference')[:2].index[0])
    if new_indices.sort_values('Difference')[:2].iloc[-1:, -1][0] == 0.0:
        pass
    else:
        lagging_sectors.append(new_indices.sort_values('Difference')[:2].index[1])


    return leading_sectors, lagging_sectors, avg_indices


def print_final_report(expiry, avg_indices, increased_activity, leading_sectors, lagging_sectors,
                       grt_avg_top_stocks, grt_avg_low_stocks, unrolled_OI_top_stocks, file_name):
    '''Function to print final report to excel'''

    if expiry == 'E':

        print_data = []

        print_data.append('----- Highlights at {0} -----'.format(expiry))

        for name in ['Nifty', 'NiftyBank', 'SSF', 'Nifty Stocks', 'Overall']:
            indices_read = pd.read_excel(output_dir + 'Report.xlsx', sheet_name='Indices')
            value = indices_read[indices_read['Indices'] == name]['Roll'].values[0]

            avg = avg_indices.loc[name].values[0]
            diff = value - avg
            if diff > 0:
                print_data.append(
                    ' *{0} rolls ({1}%) were {2} percentage points higher than the 3 month average of {3}%. '.format(
                        name, value * 100, diff * 100, avg * 100))
            else:
                print_data.append(
                    ' *{0} rolls ({1}%) were {2} percentage points lower than the 3 month average of {3}%. '.format(
                        name, value * 100, diff * -100, avg * 100))

        if expiry in ['E-5', 'E-4', 'E-3']:
            pass
        else:
            print_data.append(' *Increased roll activity was seen in {0} '.format(
                ', '.join(str(x) for x in increased_activity['BloomCode'].values)))



        print_data.append(
            '*Rolls in {0} lead their 3 month average, while those in {1} lagged '.format(leading_sectors,lagging_sectors))

        print_data.append(' *Stocks with rollovers higher than their 3 month average: ')
        i = 1

        data = OrderedDict(reversed(list(grt_avg_top_stocks.items())))
        while True:
            for key, values in data.iteritems():
                print_data.append(' {0} '.format(key))
                stocks = []
                for name in values['BloomCode'].values:
                    stocks.append("{0}({1}%)".format(str(name),
                                                     int(round(
                                                         values.set_index('BloomCode').loc[name][['SSF_rollover']][
                                                             0] * 100))))
                print_data.append(', '.join(stocks))
            i += 1
            if i > 2:
                break
            data = OrderedDict(reversed(list(grt_avg_low_stocks.items())))
            print_data.append(' *Stocks with rollovers lower than their 3 month average: ')

        pd.DataFrame(print_data).to_excel(output_dir + file_name+ 'Final.xlsx', sheet_name='Report', header=None, index=None)


    else:

        print_data = []

        print_data.append('----- Highlights at {0} -----'.format(expiry))

        for name in ['Nifty', 'NiftyBank', 'SSF', 'Nifty Stocks', 'Overall']:
            indices_read = pd.read_excel(output_dir + 'Report.xlsx', sheet_name='Indices')
            value = indices_read[indices_read['Indices'] == name]['Roll'].values[0]

            avg = avg_indices.loc[name].values[0]
            diff = value - avg
            if diff > 0:
                print_data.append(
                    ' *{0} rolls ({1}%) are {2} percentage points higher than the 3 month average of {3}%. '.format(
                        name, value * 100, diff * 100, avg * 100))
            else:
                print_data.append(
                    ' *{0} rolls ({1}%) are {2} percentage points lower than the 3 month average of {3}%. '.format(
                        name, value * 100, diff * -100, avg * 100))

        if expiry in ['E-5', 'E-4', 'E-3']:
            pass
        else:
            print_data.append(' *Increased roll activity was seen in {0} '.format(
                ', '.join(str(x) for x in increased_activity['BloomCode'].values)))

        print_data.append(
            '*Rolls in {0} names are leading their 3 month average, while those in {1} are lagging'.format(leading_sectors,lagging_sectors)
               )
        print_data.append(' *Stocks with rollovers higher than their 3 month average: ')
        i = 1

        data = OrderedDict(reversed(list(grt_avg_top_stocks.items())))
        while True:
            for key, values in data.iteritems():
                print_data.append(' {0} '.format(key))
                stocks = []
                for name in values['BloomCode'].values:
                    stocks.append("{0}({1}%)".format(str(name),
                                                     int(round(
                                                         values.set_index('BloomCode').loc[name][['SSF_rollover']][
                                                             0] * 100))))
                print_data.append(', '.join(stocks))
            i += 1
            if i > 2:
                break
            data = OrderedDict(reversed(list(grt_avg_low_stocks.items())))
            print_data.append(' *Stocks with rollovers lower than their 3 month average: ')

        print_data.append(
            ' *SSF with high unrolled OI compared to 3 mn avg: (OI in $mn and Ratio of OI to 3mn avg of expiry day delivery )')

        for key, values in OrderedDict(reversed(list(unrolled_OI_top_stocks.items()))).iteritems():
            print_data.append(' {0} '.format(key))
            stocks = []
            for name in values['BloomCode']:
                stocks.append(
                    '{0}({1}/{2}x)'.format(name, values.set_index('BloomCode').loc[name]['Million_dollar'],
                                           values.set_index('BloomCode').loc[name]['Ratio']))
            print_data.append(', '.join(stocks))

        pd.DataFrame(print_data).to_excel(output_dir + file_name+'Final.xlsx', sheet_name='Report', header=None, index=None)




def append_df_to_excel(filename, df, sheet_name='Sheet1', startrow=None,
                       **to_excel_kwargs):
    """
    Append a DataFrame [df] to existing Excel file [filename]
    into [sheet_name] Sheet.
    If [filename] doesn't exist, then this function will create it.

    Parameters:
      filename : File path or existing ExcelWriter
                 (Example: '/path/to/file.xlsx')
      df : dataframe to save to workbook
      sheet_name : Name of sheet which will contain DataFrame.
                   (default: 'Sheet1')
      startrow : upper left cell row to dump data frame.
                 Per default (startrow=None) calculate the last row
                 in the existing DF and write to the next row...
      to_excel_kwargs : arguments which will be passed to `DataFrame.to_excel`
                        [can be dictionary]

    Returns: None
    """
    from openpyxl import load_workbook

    # ignore [engine] parameter if it was passed
    if 'engine' in to_excel_kwargs:
        to_excel_kwargs.pop('engine')

    # create a writer for this month and year
    writer = pd.ExcelWriter(master_dir+filename, engine='openpyxl')

    try:
        # try to open an existing workbook
        writer.book = load_workbook(master_dir+filename)
        # get the last row in the existing Excel sheet
        # if it was not specified explicitly
        if not startrow and sheet_name in writer.book.sheetnames:
            startrow = writer.book[sheet_name].max_row

        # copy existing sheets
        writer.sheets = dict(
            (ws.title, ws) for ws in writer.book.worksheets)
    except FileNotFoundError:
        # file does not exist yet, we will create it
        pass

    if not startrow:
        startrow = 0

    # write out the new sheet
    df.to_excel(writer, sheet_name, startrow=startrow, **to_excel_kwargs)

    # save the workbook
    writer.save()



# Remove duplicate rows from master data
def drop_duplicate_rows(filename):
    '''Function to drop duplicate rows from master dumps'''

    df = pd.read_excel(master_dir+filename, sheet_name= 'Sheet1')
    df.drop_duplicates(keep='first', inplace=True)
    df.to_excel(master_dir+filename, index = False)







In [308]:

#import xlwings as xw
#import numpy as np
import pandas as pd
#from collections import OrderedDict
from time import time
#from dateutil import parser
import os
import Utility_functions


master_dir = 'X:\\Data_Analytics\\ETL\\Master\\'
download_dir = 'X:\\Data_Analytics\\ETL\\Download\\'
output_dir = 'X:\\Data_Analytics\\ETL\\Output\\'


def main_processor(expiry, month, year, file_name,name):
    # if valid Expiry date process and generate reports
    # Remove duplicate rows if any in master files
    Utility_functions.drop_duplicate_rows('RollData.xlsx')
    Utility_functions.drop_duplicate_rows('Indices_dump.xlsx')
    Utility_functions.drop_duplicate_rows('Unrolled_dumped.xlsx')

    # Dataframe with product far, near and rollover values
    data, master = Utility_functions.bhavcopy_processor(file_name)
    result = Utility_functions.sector_rollovers(data)

    # Dump data
    dumper, indices_rollover, unrolled_dumper = Utility_functions.rollover_data_dumper(data, master, result, expiry,
                                                                                       month, year, name)

    # get top and low stocks from grt than avg sheet
    grt_avg_top_stocks, grt_avg_low_stocks = Utility_functions.greaterthanAverage(expiry, month)
    # get top unrolled OI stocks
    unrolled_OI_top_stocks = {}
    if expiry == 'E':
        pass
    else:
        # calculate unrolled OI
        unrolled_OI_top_stocks = Utility_functions.unrolled_OI_Calculator(expiry, month)
    # increased activity
    increased_activity = []
    if expiry in ['E-5', 'E-4', 'E-3']:
        pass
    else:
        # calculate increased activity
        increased_activity = Utility_functions.increased_activity_calc(expiry, month, year)

    # get top 2 leading and lagging sectors
    leading_sectors, lagging_sectors, avg_indices = Utility_functions.leading_lagging_sectors(expiry, month)

    # call final print
    Utility_functions.print_final_report(expiry, avg_indices, increased_activity, leading_sectors, lagging_sectors,
                                         grt_avg_top_stocks, grt_avg_low_stocks, unrolled_OI_top_stocks, name)

    # Update master files
    Utility_functions.append_df_to_excel('RollData.xlsx', df=dumper, header=None, index=False)
    Utility_functions.append_df_to_excel('Indices_dump.xlsx', df=indices_rollover, header=None, index=False)
    Utility_functions.append_df_to_excel('Unrolled_dumped.xlsx', df=unrolled_dumper, header=None, index=False)





def main():

    # traverse through all bhavcopies
    for r, d, f in os.walk(download_dir):
        for csv_file in f:
            file_name = csv_file

            # extract date from input bhavcopy name
            name = file_name[2:11]
            date, month, year = name[:2], name[2:5], name[5:]

            # read expiry from master dump to get expiry code
            # if not an expiry throw exception
            try:
                dates = pd.read_excel(master_dir + 'Expiry_dates_master.xlsx')
                expiry = str(dates[(dates['Date'] == int(date))
                                   & (dates['Month'] == month) & (dates['Year'] == int(year))]['Expiry'].values[0])
                main_processor(expiry, month, year, file_name,name)
                break
            except IndexError:
                print 'Exception: {0}/{1}/{2} - Not a valid expiry date...'.format(date, month, year)





start_time = time()

if __name__ == '__main__':
    main()

end_time = time()


print "Execution time: {0} Seconds.... ".format(end_time - start_time)


Exception: 12/SEP/2018 - Not a valid expiry date...
Dumping data for E-3 OCT 2018.....
Execution time: 26.986000061 Seconds.... 


In [390]:
# read from dumper object for SSF rolldata
rolldata = pd.read_excel(master_dir + 'RollData.xlsx')
rolldata = rolldata.set_index('SYMBOL')

# average is appended in sorted manner
stock_names = list(sorted(set(rolldata.index)))
average_ssf = pd.DataFrame(average_calculator(expiry, 'SSF_rollover', stock_names, rolldata, month), index=stock_names,
                           columns=['Average'])
average_ssf.index.name = 'SYMBOL'


# Greater than average
# read from dumped data
grtthanaverage = pd.read_excel(output_dir + 'Report.xlsx', sheet_name='SSF')

grtthanaverage = grtthanaverage.merge(average_ssf, on='SYMBOL')

# difference of today's rollover and average of last 3 months
grtthanaverage['Difference'] = grtthanaverage['SSF_rollover'] - grtthanaverage['Average']

# read master for fetching type of stock nifty, non-nifty and physical
type_of_stock = pd.read_excel(master_dir + 'MasterData.xlsx', sheet_name='Physical')
grtthanaverage = grtthanaverage.merge(type_of_stock, on='SYMBOL')

# pass False for top performing as sorting parameter
grt_avg_top_stocks = sort_stocks(5, False, grtthanaverage, 'Difference')

# pass True for low performing as sorting parameter
grt_avg_low_stocks = sort_stocks(5, True, grtthanaverage, 'Difference')

    


In [392]:
for key, values in grt_avg_low_stocks.items():
    grt_avg_low_stocks[key] = values.sort_values(by=['SSF_rollover']).head(5)
print grt_avg_low_stocks

{u'Physical':     Expiry      SYMBOL BloomCode Month  Year  SSF_rollover  Average  \
113    E-3        OFSS      OFSS   OCT  2018         0.055     0.24   
94     E-3        KSCL      KSCL   OCT  2018         0.089     0.34   
135    E-3         SRF       SRF   OCT  2018         0.169     0.36   
182    E-3         MGL     MAHGL   OCT  2018         0.185     0.41   
184    E-3  CANFINHOME      CANF   OCT  2018         0.218     0.48   

     Difference      TYPE  
113      -0.185  Physical  
94       -0.251  Physical  
135      -0.191  Physical  
182      -0.225  Physical  
184      -0.262  Physical  , u'Non Nifty':     Expiry    SYMBOL BloomCode Month  Year  SSF_rollover  Average  Difference  \
187    E-3       MCX       MCX   OCT  2018         0.088     0.36      -0.272   
142    E-3  TATACOMM      TCOM   OCT  2018         0.114     0.29      -0.176   
83     E-3       IRB       IRB   OCT  2018         0.144     0.32      -0.176   
129    E-3  RELINFRA      RELI   OCT  2018         0

In [397]:
list1 = ['One','Two']

In [398]:
list1.pop()

'Two'

In [409]:


def main():
    book = openpyxl.Workbook()
    
    sheet = book.worksheets[0]
    # Writing to sheet
    sheet.cell(row=2, column=1).value = 'sample'
    sheet.cell(row=4, column=1).value = 'sample1'
    sheet.cell(row=6, column=1).value = 'sample2'
    sheet.cell(row=8, column=1).value = 'sample3'
    sheet.cell(row=10, column=1).value = 'sample4'
    book.save('Sample.xlsx')

if __name__ == '__main__':
    main()

In [411]:
import pandas 

In [413]:
pandas.__version__

u'0.23.0'

In [4]:
import xlwings as xw
import numpy as np
import pandas as pd
from collections import OrderedDict
from time import time
from dateutil import parser
import os
import openpyxl
import sys

master_dir = 'X:\\Data_Analytics\\ETL\\Master\\'
download_dir = 'X:\\Data_Analytics\\ETL\\Download\\'
output_dir = 'X:\\Data_Analytics\\ETL\\Output\\'


# calculate SSF roll over

def ssf_calculator(a, b):
    try:
        return round(b / (a + b), 3)
    except:
        return 0.0


def bhavcopy_processor(file_name):
    '''Function to read and process bhavcopy
    Convert it into a pivot_table
    calculate product-far/ product-near and return that dataframe '''

    # read bhav copy as workbook
    wb = xw.Book(download_dir + file_name)

    # read sheet
    sht = wb.sheets[file_name[:-4]]

    # convert sheet to pandas dataframe
    all_stocks = sht.range('A1').options(pd.DataFrame, expand='table').value

    # select only FUTSTK and FUTIDX
    new_index = ['FUTSTK', 'FUTIDX']
    all_stocks = all_stocks.loc[new_index]

    # Categorization based on master document
    # read master doc
    mast_book = xw.Book(master_dir + 'MasterData.xlsx')

    # read sheet
    mast_sheet = mast_book.sheets('Sheet1')

    # convert sheet to pandas dataframe
    master = mast_sheet.range('A1').options(pd.DataFrame, expand='table').value
    # master.rename(columns={'SymbolName': 'SYMBOL'}, inplace=True)

    # filter master for symbol, bloomcode and sector
    master.set_index(keys='SYMBOL', inplace=True)
    master.drop(columns=['IsActiveFNO'], axis=1, inplace=True)

    # join dataframes master and all_stocks based on SYMBOL
    all_stocks = all_stocks.join(master, on='SYMBOL')

    # create Pivot table
    all_stocks = all_stocks.pivot_table(values=['OPEN_INT', 'SETTLE_PR'], index=['SYMBOL', 'Sector', 'IsNifty', 'Type'],
                                        columns='EXPIRY_DT', aggfunc={'SETTLE_PR': np.sum,
                                                                      'OPEN_INT': np.sum})

    # calculate product-near and product-far
    data = pd.DataFrame([map(round, all_stocks['OPEN_INT'].iloc[:, 0] * all_stocks['SETTLE_PR'].iloc[:, 0])
                            , map(round, all_stocks['OPEN_INT'].iloc[:, 1] * all_stocks['SETTLE_PR'].iloc[:, 1] +
                                  all_stocks['OPEN_INT'].iloc[:, 2] * all_stocks['SETTLE_PR'].iloc[:, 2])]).T

    data.columns = ['Product-Near', 'Product-Far']
    data.index = all_stocks.index

    # apply roll over to all stocks and append ssf_rollover
    data['SSF_rollover'] = data.apply(lambda row: ssf_calculator(row['Product-Near'], row['Product-Far']), axis=1)
    pd.set_option('display.precision', 10)
    return data, master


def sector_rollovers(data):
    '''Function to calculate sector vice rollovers and dump to base Report_stats.xlsx'''

    # Group all sectors and run sum() aggregate
    Sector = data.groupby('Sector').sum()
    Sector['Roll'] = Sector.apply(lambda row: ssf_calculator(row['Product-Near'], row['Product-Far']), axis=1)
    Sector.drop(columns='SSF_rollover', inplace=True, axis=1)

    # append nifty stocks to Sector table
    nifty_product_near = data.groupby('IsNifty').get_group(True).sum()['Product-Near']
    nifty_product_far = data.groupby('IsNifty').get_group(True).sum()['Product-Far']
    nifty_roll = ssf_calculator(nifty_product_near, nifty_product_far)

    Nifty_Stocks_df = pd.DataFrame(
        {'Product-Near': [nifty_product_near], 'Product-Far': [nifty_product_far], 'Roll': [nifty_roll]},
        index=['Nifty Stocks'])

    # append non nifty stocks to sector table
    non_nifty_product_near = data.groupby('IsNifty').get_group(False).groupby('Type').sum()['Product-Near']['SSF']
    non_nifty_product_far = data.groupby('IsNifty').get_group(False).groupby('Type').sum()['Product-Far']['SSF']
    non_nifty_roll = ssf_calculator(non_nifty_product_near, non_nifty_product_far)

    Non_Nifty_Stocks_df = pd.DataFrame(
        {'Product-Near': [non_nifty_product_near], 'Product-Far': [non_nifty_product_far], 'Roll': [non_nifty_roll]},
        index=['Non-Nifty Stocks'])

    # append SSF stock to sector table
    ssf_df = pd.DataFrame({'Product-Near': data.groupby('Type').sum().loc['SSF']['Product-Near'],
                           'Product-Far': data.groupby('Type').sum().loc['SSF']['Product-Far'],
                           'Roll': ssf_calculator(data.groupby('Type').sum().loc['SSF']['Product-Near'],
                                                  data.groupby('Type').sum().loc['SSF']['Product-Far'])},
                          index=['SSF'])

    # append NIFTY to sector table
    NIFTY_df = pd.DataFrame({'Product-Near': data.loc['NIFTY'].loc[:, 'Product-Near'][0],
                             'Product-Far': data.loc['NIFTY'].loc[:, 'Product-Far'][0],
                             'Roll': ssf_calculator(data.loc['NIFTY'].loc[:, 'Product-Near'][0],
                                                    data.loc['NIFTY'].loc[:, 'Product-Far'][0])},
                            index=['Nifty'])

    # append BANKNIFTY to sector table
    BANKNIFTY_df = pd.DataFrame({'Product-Near': data.loc['BANKNIFTY'].loc[:, 'Product-Near'][0],
                                 'Product-Far': data.loc['BANKNIFTY'].loc[:, 'Product-Far'][0],
                                 'Roll': ssf_calculator(data.loc['BANKNIFTY'].loc[:, 'Product-Near'][0],
                                                        data.loc['BANKNIFTY'].loc[:, 'Product-Far'][0])},
                                index=['NiftyBank'])

    Overall = pd.DataFrame({'Product-Near': data['Product-Near'].sum(),
                            'Product-Far': data['Product-Far'].sum(),
                            'Roll': round(
                                data['Product-Far'].sum() / (data['Product-Near'].sum() + data['Product-Far'].sum()),
                                2)},
                           index=['Overall'])

    frames = [Sector, Nifty_Stocks_df, Non_Nifty_Stocks_df, ssf_df, NIFTY_df, BANKNIFTY_df, Overall]
    result = pd.concat(frames, sort=False)
    result.sort_index(inplace=True)
    result.drop(index=['None', 'Index'], axis=0, inplace=True)

    # write dataframe to excel
    writer = pd.ExcelWriter(output_dir + 'Report_stats.xlsx', engine='xlsxwriter')
    result.to_excel(writer, sheet_name='SSF & Indices Report')
    return result


def rollover_data_dumper(data, master, result, expiry, month, year, file_name):
    '''Function to dump rollover data for expiry'''
    print 'Dumping data for {} {} {}.....'.format(expiry, month, year)

    # ignore index i.e consider only SSF stocks

    ssf_rollover = data.groupby('Type').get_group('SSF').reset_index(level=[1, 2, 3], drop=True)[['SSF_rollover']]

    # fetch bloomcode from master and join dataframes to dumper

    stock_bloom = master.groupby('Type').get_group('SSF')[['BloomCode']]
    dumper = stock_bloom.merge(ssf_rollover, on='SYMBOL')

    # dump expiry, month and year

    dumper['Expiry'], dumper['Month'], dumper['Year'] = expiry, month, year
    dumper.reset_index(inplace=True)

    # reaarange according to schema

    dumper = dumper[['Expiry', 'SYMBOL', 'BloomCode', 'Month', 'Year', 'SSF_rollover']]
    writer = pd.ExcelWriter(output_dir + 'Report.xlsx', engine='xlsxwriter')
    dumper.to_excel(writer, sheet_name='SSF', index=False)

    # dump indices rollover

    indices_rollover = pd.DataFrame(result['Roll'])
    indices_rollover['Expiry'], indices_rollover['Month'], indices_rollover['Year'] = expiry, month, year
    indices_rollover.index.name = 'Indices'
    indices_rollover.reset_index(inplace=True)
    indices_rollover = indices_rollover[['Expiry', 'Month', 'Year', 'Indices', 'Roll']]
    indices_rollover.to_excel(writer, sheet_name='Indices', index=False)

    # Unrolled data

    unrolled = master.groupby('Type').get_group('SSF')[['BloomCode']].merge(
        data.groupby('Type').get_group('SSF').reset_index(
            level=[1, 2, 3], drop=True)[['Product-Near']], on='SYMBOL')

    # Convert product-near to million dollars rates

    dollar = pd.read_excel(master_dir + 'USDINRRates.xlsx')
    dollar['Date'] = dollar.apply(lambda row: parser.parse(row['Date']), axis=1)
    dollar.set_index('Date', inplace=True)

    # extract date from bhavcopy filename and convert it to date object
    # traverse in dollar dump for that date and retrive its US dollar rate

    dollar_value = dollar.loc[file_name]['Rate'][0]

    unrolled['Million_dollar'] = unrolled['Product-Near'].apply(lambda x: round(x / (dollar_value * 1000000), 1))

    # dump million dollar rates of stocks

    unrolled_dumper = master.groupby('Type').get_group('SSF')[['BloomCode']]
    unrolled_dumper['Expiry'], unrolled_dumper['Month'], unrolled_dumper['Year'] = expiry, month, year
    unrolled_dumper['Million_dollar'] = unrolled['Million_dollar']
    unrolled_dumper.reset_index(inplace=True)
    unrolled_dumper = unrolled_dumper[['Expiry', 'SYMBOL', 'BloomCode', 'Month', 'Year', 'Million_dollar']]

    unrolled_dumper.to_excel(writer, sheet_name='Unrolled', index=False)

    return dumper, indices_rollover, unrolled_dumper


# Calculate avergae of last three months
def average_calculator(expiry, value, stock_names, data_df, month):
    '''Function to calculate average of last three months'''
    months = {"JAN":0, "FEB":1, "MAR":2, "APR":3, "MAY":4, "JUN":5, "JUL":6, "AUG":7, "SEP":8, "OCT":9, "NOV":10, "DEC":11 }
    #months.keys()[months.values().index(months[month]-1)]
    average = []


    for i in range(len(stock_names)):
        # average.append( round(rolldata.groupby('Expiry').get_group(expiry).loc[stock_names[i]][-3:]['SSF_rollover'].mean(), 2) )
        average.append(round(data_df[(data_df['Expiry']==expiry) &
         ((data_df['Month']==months.keys()[months.values().index(months[month]-1)]) |
        (data_df['Month']==months.keys()[months.values().index(months[month]-2)]) |
        (data_df['Month']==months.keys()[months.values().index(months[month]-3)])) ].loc[stock_names[i]][value].mean(), 2))

    return average


# Sort elements based on criteria top or low performing stocks
def sort_stocks(rank_order, sorting_type, data, criteria):
    sorted_stocks = {}

    for group_name, group_elements in data.groupby('TYPE'):
        sorted_stocks[group_name] = group_elements.sort_values(ascending=sorting_type, by=criteria)[: rank_order]

    return sorted_stocks


def greaterthanAverage(expiry, month):
    '''Function for greater than average sheet'''

    # read from dumper object for SSF rolldata
    rolldata = pd.read_excel(master_dir + 'RollData.xlsx')
    rolldata = rolldata.set_index('SYMBOL')

    # average is appended in sorted manner
    stock_names = list(sorted(set(rolldata.index)))
    average_ssf = pd.DataFrame(average_calculator(expiry, 'SSF_rollover', stock_names, rolldata, month), index=stock_names,
                               columns=['Average'])
    average_ssf.index.name = 'SYMBOL'


    # Greater than average
    # read from dumped data
    grtthanaverage = pd.read_excel(output_dir + 'Report.xlsx', sheet_name='SSF')

    grtthanaverage = grtthanaverage.merge(average_ssf, on='SYMBOL')

    # difference of today's rollover and average of last 3 months
    grtthanaverage['Difference'] = grtthanaverage['SSF_rollover'] - grtthanaverage['Average']

    # read master for fetching type of stock nifty, non-nifty and physical
    type_of_stock = pd.read_excel(master_dir + 'MasterData.xlsx', sheet_name='Physical')
    grtthanaverage = grtthanaverage.merge(type_of_stock, on='SYMBOL')

    # pass False for top performing as sorting parameter
    grt_avg_top_stocks = sort_stocks(5, False, grtthanaverage, 'Difference')
    for key, values in grt_avg_top_stocks.items():
        grt_avg_top_stocks[key] = values.sort_values(ascending=False, by=['SSF_rollover']).head(5)

    # pass True for low performing as sorting parameter
    grt_avg_low_stocks = sort_stocks(5, True, grtthanaverage, 'Difference')
    for key, values in grt_avg_low_stocks.items():
        grt_avg_low_stocks[key] = values.sort_values(ascending= True, by=['SSF_rollover']).head(5)

    return grt_avg_top_stocks, grt_avg_low_stocks


def del_qty_average():
    '''Function to calculate last 3months del quantity average'''

    # read del qty dump
    del_qty = pd.read_excel(master_dir + 'Del Qty.xlsx')
    # parse del qty dates to date format
    del_qty['Date'] = del_qty.apply(lambda row: parser.parse(row['Date']), axis=1)
    del_qty.set_index('SYMBOL', inplace=True)
    stock_names = list(set(del_qty.index))
    average = {}
    for name in stock_names:
        average[name] = round(del_qty.loc[name].sort_values(ascending=False, by='Date').iloc[:3, -1].mean(),2)

    average_del = pd.DataFrame(average.items(), columns=['SYMBOL', 'Del Avg'])
    average_del.dropna(inplace=True, axis=0)
    average_del.set_index('SYMBOL', inplace=True)
    return average_del


def unrolled_OI_Calculator(expiry, month):
    '''Function to compute unrolled OI sheet'''
    # unrolled OI calculator

    unrolled_today = pd.read_excel(master_dir + 'Unrolled_dumped.xlsx')
    unrolled_today = unrolled_today.set_index('SYMBOL')

    # average is appended in sorted manner
    stock_names = list(sorted(set(unrolled_today.index)))

    average_unrolled = pd.DataFrame(average_calculator(expiry, 'Million_dollar', stock_names, unrolled_today, month),
                                    index=stock_names, columns=['Average'])
    average_unrolled.index.name = 'SYMBOL'

    # read unrolled (million_dollar )from dumped data
    unrolled_OI_calc = pd.read_excel(output_dir + 'Report.xlsx', sheet_name='Unrolled')
    unrolled_OI_calc = unrolled_OI_calc.merge(average_unrolled, on='SYMBOL')


    # difference of today's million_unrolled and average of last 3 months
    unrolled_OI_calc['Difference'] = unrolled_OI_calc['Million_dollar'] - unrolled_OI_calc['Average']

    # Calculate percent for unrolled- avg/ avg
    unrolled_OI_calc['Percent'] = unrolled_OI_calc.apply(
        lambda row: round((row['Million_dollar'] - row['Average']) / row['Average'] * 100), axis=1)

    # fetch del qty averages for last three months
    del_qty = del_qty_average()
    unrolled_OI_calc = unrolled_OI_calc.merge(del_qty, on='SYMBOL')
    unrolled_OI_calc['Ratio'] = unrolled_OI_calc['Million_dollar'] / unrolled_OI_calc['Del Avg']
    unrolled_OI_calc['Ratio'] = unrolled_OI_calc['Ratio'].apply(lambda x: round(x))

    # read master for fetching type of stock nifty, non-nifty and physical
    type_of_stock = pd.read_excel(master_dir + 'MasterData.xlsx', sheet_name='Physical')
    unrolled_OI_calc = unrolled_OI_calc.merge(type_of_stock, on='SYMBOL')

    # pass False for top performing as sorting parameter (take only top stocks for unrolled)
    unrolled_OI_top_stocks = sort_stocks(7, False, unrolled_OI_calc, 'Percent')

    # sort on ratio and million dollar values
    for key, values in unrolled_OI_top_stocks.items():
        unrolled_OI_top_stocks[key] = values.sort_values(ascending=False, by=['Ratio', 'Million_dollar']).head(4)

    return unrolled_OI_top_stocks


def increased_activity_calc(expiry, month, year):
    # increased Activity
    # read dumped value for SSF stucks for current expiry

    increased_activity = pd.read_excel(output_dir + 'Report.xlsx', sheet_name='SSF')
    increased_activity.set_index('SYMBOL', inplace=True)
    # read SSF stocks for T-1 expiry
    expiry_list = 'E-5 E-4 E-3 E-2 E-1 E'.split(' ')

    previous_day_expiry = pd.read_excel(master_dir + 'RollData.xlsx')
    previous_day_expiry = previous_day_expiry[
        (previous_day_expiry['Expiry'] == expiry_list[expiry_list.index(expiry) - 1]) & (
                previous_day_expiry['Month'] == month) &
        (previous_day_expiry['Year'] == int(year))][['SYMBOL', 'SSF_rollover']]
    previous_day_expiry.set_index('SYMBOL', inplace=True)

    # merge on symbol

    increased_activity = increased_activity.merge(previous_day_expiry, on='SYMBOL')
    increased_activity.columns = ['Expiry', 'BloomCode', 'Month', 'Year', 'SSF_rollover_T', 'SSF_rollover_T-1']

    # calculate difference Today - (T-1)

    increased_activity['Difference'] = increased_activity['SSF_rollover_T'] - increased_activity['SSF_rollover_T-1']
    increased_activity = increased_activity.sort_values(ascending=False, by='Difference').head(10)
    return increased_activity


def leading_lagging_sectors(expiry, month):
    '''Function to fetch leading and lagging sectors'''

    # read dumped indices
    dumped_indices = pd.read_excel(master_dir + 'Indices_dump.xlsx')
    dumped_indices = dumped_indices.groupby('Expiry').get_group(expiry)
    indices_names = list(sorted(set(dumped_indices['Indices'])))
    dumped_indices.set_index('Indices', inplace=True)

    # Average of indicies for last 3 months
    avg_indices = pd.DataFrame(average_calculator(expiry, 'Roll', indices_names, dumped_indices, month),
                               index=indices_names, columns=['Average'])
    avg_indices.index.name = 'Indices'



    indices_read = pd.read_excel(output_dir + 'Report.xlsx', sheet_name='Indices')
    indices_diff = indices_read.set_index('Indices')
    indices_diff = indices_diff.merge(avg_indices, on='Indices')
    indices_diff['Difference'] = indices_diff['Roll'] - indices_diff['Average']
    indices_diff['Difference'] = indices_diff.apply(lambda row: round(row['Difference'],2), axis=1)
    # Sectors to ignore
    exclude_list = ['Nifty', 'NiftyBank', 'Nifty Stocks', 'Non-Nifty Stocks', 'SSF', 'Overall', 'Real Estate',
                    'Diversified', 'chemicals', 'Media']
    new_indices = indices_diff.drop(index=exclude_list)
    leading_sectors = []
    lagging_sectors = []
    # append leading and lagging sectors
    #topmost element
    if new_indices.sort_values('Difference', ascending = False)[:2].iloc[:-1,-1][0] == 0.0:
        pass
    else:
        leading_sectors.append(new_indices.sort_values('Difference', ascending = False)[:2].index[0])
    #2nd element in top stocks
    if new_indices.sort_values('Difference', ascending = False)[:2].iloc[-1:, -1][0] == 0.0:
        pass
    else:
        leading_sectors.append(new_indices.sort_values('Difference', ascending = False)[:2].index[1])
    #lowest stock
    if new_indices.sort_values('Difference')[:2].iloc[:-1,-1][0] == 0.0:
        pass
    else:
        lagging_sectors.append(new_indices.sort_values('Difference')[:2].index[0])
    #2nd last stock
    if new_indices.sort_values('Difference')[:2].iloc[-1:, -1][0] == 0.0:
        pass
    else:
        lagging_sectors.append(new_indices.sort_values('Difference')[:2].index[1])


    return leading_sectors, lagging_sectors, avg_indices


def print_final_report(expiry, avg_indices, increased_activity, leading_sectors, lagging_sectors,
                       grt_avg_top_stocks, grt_avg_low_stocks, unrolled_OI_top_stocks, file_name):
    '''Function to print final report to excel'''

    # initialize the workbook
    book = openpyxl.Workbook()
    # get the default worksheet
    sheet = book.worksheets[0]
    # write to sheet one by one and save

    sheet.cell(row=2, column=1).value = '----- Highlights at {0} -----'.format(expiry)

    if expiry == 'E':


        i = 0
        for name in ['Nifty', 'NiftyBank', 'SSF', 'Nifty Stocks', 'Overall']:
            indices_read = pd.read_excel(output_dir + 'Report.xlsx', sheet_name='Indices')
            value = indices_read[indices_read['Indices'] == name]['Roll'].values[0]

            avg = avg_indices.loc[name].values[0]
            diff = value - avg
            if diff > 0:
                sheet.cell(row=4+i, column=1).value = ' *{0} rolls ({1}%) were {2} percentage points higher than the 3 month average of {3}%. '.format(
                    name, value * 100, diff * 100, avg * 100)
                i += 2
            else:
                sheet.cell(row=4 + i, column=1).value =' *{0} rolls ({1}%) were {2} percentage points lower than the 3 month average of {3}%. '.format(
                    name, value * 100, diff * -100, avg * 100)
                i += 2

        if expiry in ['E-5', 'E-4', 'E-3']:
            pass
        else:
            sheet.cell(row=4 + i, column=1).value = ' *Increased roll activity was seen in {0} '.format(
                ', '.join(str(x) for x in increased_activity['BloomCode'].values))
            i += 2


        if len(leading_sectors) == 0 and len(lagging_sectors) == 0:
            pass
        elif len(leading_sectors) == 1 and len(lagging_sectors) == 0:
            sheet.cell(row=4+i, column=1).value ='*Rolls in {0} lead their 3 month average '.format(leading_sectors.pop())
            i += 2
        elif len(leading_sectors) == 0 and len(lagging_sectors) == 1:
            sheet.cell(row=4+i, column=1).value ='*Rolls in {0} lagged their 3 month average '.format(lagging_sectors.pop())
            i += 2
        elif len(leading_sectors) == 2 and len(lagging_sectors) == 1:
            sheet.cell(row=4+i, column=1).value ='*Rolls in {0} and {} lead their 3 month average, while those in {3} lagged their 3 month average '.format(leading_sectors.pop(),
                                                                                                                                                            leading_sectors.pop(),
                                                                                                                                                            lagging_sectors.pop())
            i += 2
        elif len(leading_sectors) == 1 and len(lagging_sectors) == 2:
            sheet.cell(row=4+i, column=1).value ='*Rolls in {0} lead their 3 month average, while those in {1} and {2} lagged their 3 month average '.format(leading_sectors.pop(),
                                                                                                                                                             lagging_sectors.pop(),
                                                                                                                                                             lagging_sectors.pop())
            i += 2
        elif len(leading_sectors) == 2 and len(lagging_sectors) == 2:
            sheet.cell(row=4+i, column=1).value ='*Rolls in {0} and {1} lead their 3 month average, while those in {2} and {3} lagged their 3 month average '.format(leading_sectors.pop(),
                                                                                                                                                                     leading_sectors.pop(),
                                                                                                                                                                     lagging_sectors.pop(),
                                                                                                                                                                     lagging_sectors.pop())
            i += 2



        sheet.cell(row=4+i, column=1).value =' *Stocks with rollovers higher than their 3 month average: '
        j = 4+i+1

        i = 1

        data = OrderedDict(reversed(list(grt_avg_top_stocks.items())))
        while True:
            for key, values in data.iteritems():
                sheet.cell(row=j, column=1).value =' {0} '.format(key); j += 1
                stocks = []
                for name in values['BloomCode'].values:
                    stocks.append("{0}({1}%)".format(str(name),
                                                     int(round(
                                                         values.set_index('BloomCode').loc[name][['SSF_rollover']][
                                                             0] * 100))))
                sheet.cell(row=j, column=1).value =', '.join(stocks); j+= 1
            i += 1
            if i > 2:
                break
            data = OrderedDict(reversed(list(grt_avg_low_stocks.items())))
            j += 1
            sheet.cell(row=j, column=1).value =' *Stocks with rollovers lower than their 3 month average: '; j+=1

        book.save(output_dir+file_name+'Final.xlsx')
        #pd.DataFrame(print_data).to_excel(output_dir + file_name+ 'Final.xlsx', sheet_name='Report', header=None, index=None)


    else:


        i=0

        for name in ['Nifty', 'NiftyBank', 'SSF', 'Nifty Stocks', 'Overall']:
            indices_read = pd.read_excel(output_dir + 'Report.xlsx', sheet_name='Indices')
            value = indices_read[indices_read['Indices'] == name]['Roll'].values[0]

            avg = avg_indices.loc[name].values[0]
            diff = value - avg
            if diff > 0:
                sheet.cell(row=4 + i, column=1).value =' *{0} rolls ({1}%) are {2} percentage points higher than the 3 month average of {3}%. '.format(
                    name, value * 100, diff * 100, avg * 100)
                i += 2
            else:
                sheet.cell(row=4 + i, column=1).value =' *{0} rolls ({1}%) are {2} percentage points lower than the 3 month average of {3}%. '.format(
                    name, value * 100, diff * -100, avg * 100)
                i += 2

        if expiry in ['E-5', 'E-4', 'E-3']:
            pass
        else:
            sheet.cell(row=4+i, column=1).value =' *Increased roll activity was seen in {0} '.format(
                ', '.join(str(x) for x in increased_activity['BloomCode'].values))
            i += 2

        # leading and lagging print stmt
        if len(leading_sectors) == 0 and len(lagging_sectors) == 0:
            pass
        elif len(leading_sectors) == 1 and len(lagging_sectors) == 0:
            sheet.cell(row=4 + i, column=1).value ='*Rolls in {0} are leading their 3 month average '.format(leading_sectors.pop())
            i += 2
        elif len(leading_sectors) == 0 and len(lagging_sectors) == 1:
            sheet.cell(row=4 + i, column=1).value ='*Rolls in {0} are lagging their 3 month average '.format(lagging_sectors.pop())
            i += 2
        elif len(leading_sectors) == 2 and len(lagging_sectors) == 1:
            sheet.cell(row=4+i, column=1).value ='*Rolls in {0} and {} are leading their 3 month average, while those in {3} are lagging their 3 month average '.format(leading_sectors.pop(),
                                                                                                                                                                        leading_sectors.pop(),
                                                                                                                                                                        lagging_sectors.pop())
            i += 2
        elif len(leading_sectors) == 1 and len(lagging_sectors) == 2:
            sheet.cell(row=4+i, column=1).value ='*Rolls in {0} are leading their 3 month average, while those in {1} and {2} are lagging their 3 month average '.format(leading_sectors.pop(),
                                                                                                                                                                         lagging_sectors.pop(),
                                                                                                                                                                         lagging_sectors.pop())
            i += 2
        elif len(leading_sectors) == 2 and len(lagging_sectors) == 2:
            sheet.cell(row=4+i, column=1).value ='*Rolls in {0} and {1} are leading their 3 month average, while those in {2} and {3} are lagging their 3 month average '.format(leading_sectors.pop(),
                                                                                                                                                                                 leading_sectors.pop(),
                                                                                                                                                                                 lagging_sectors.pop(),
                                                                                                                                                                                 lagging_sectors.pop())
            i += 2


        sheet.cell(row=4+i, column=1).value =' *Stocks with rollovers higher than their 3 month average: '
        j = 4+i+1
        i = 1

        data = OrderedDict(reversed(list(grt_avg_top_stocks.items())))
        while True:
            for key, values in data.iteritems():
                sheet.cell(row=j, column=1).value =' {0} '.format(key); j += 1
                stocks = []
                for name in values['BloomCode'].values:
                    stocks.append("{0}({1}%)".format(str(name),
                                                     int(round(
                                                         values.set_index('BloomCode').loc[name][['SSF_rollover']][
                                                             0] * 100))))
                sheet.cell(row=j, column=1).value =', '.join(stocks);j += 1
            i += 1
            if i > 2:
                break
            data = OrderedDict(reversed(list(grt_avg_low_stocks.items())))
            j+=1
            sheet.cell(row=j, column=1).value =' *Stocks with rollovers lower than their 3 month average: '; j += 1

        j+=1
        sheet.cell(row=j, column=1).value =' *SSF with high unrolled OI compared to 3 mn avg: (OI in $mn and Ratio of OI to 3mn avg of expiry day delivery )'
        j += 1

        for key, values in OrderedDict(reversed(list(unrolled_OI_top_stocks.items()))).iteritems():
            sheet.cell(row=j, column=1).value =' {0} '.format(key); j += 1
            stocks = []
            for name in values['BloomCode']:
                stocks.append(
                    '{0}({1}/{2}x)'.format(name, values.set_index('BloomCode').loc[name]['Million_dollar'],
                                           values.set_index('BloomCode').loc[name]['Ratio']))
            sheet.cell(row=j, column=1).value =', '.join(stocks); j += 1


        book.save(output_dir + file_name + 'Final.xlsx')
        #pd.DataFrame(print_data).to_excel(output_dir + file_name+'Final.xlsx', sheet_name='Report', header=None, index=None)




def append_df_to_excel(filename, df, sheet_name='Sheet1', startrow=None,
                       **to_excel_kwargs):
    """
    Append a DataFrame [df] to existing Excel file [filename]
    into [sheet_name] Sheet.
    If [filename] doesn't exist, then this function will create it.

    Parameters:
      filename : File path or existing ExcelWriter
                 (Example: '/path/to/file.xlsx')
      df : dataframe to save to workbook
      sheet_name : Name of sheet which will contain DataFrame.
                   (default: 'Sheet1')
      startrow : upper left cell row to dump data frame.
                 Per default (startrow=None) calculate the last row
                 in the existing DF and write to the next row...
      to_excel_kwargs : arguments which will be passed to `DataFrame.to_excel`
                        [can be dictionary]

    Returns: None
    """
    from openpyxl import load_workbook

    # ignore [engine] parameter if it was passed
    if 'engine' in to_excel_kwargs:
        to_excel_kwargs.pop('engine')

    # create a writer for this month and year
    writer = pd.ExcelWriter(master_dir+filename, engine='openpyxl')

    try:
        # try to open an existing workbook
        writer.book = load_workbook(master_dir+filename)
        # get the last row in the existing Excel sheet
        # if it was not specified explicitly
        if not startrow and sheet_name in writer.book.sheetnames:
            startrow = writer.book[sheet_name].max_row

        # copy existing sheets
        writer.sheets = dict(
            (ws.title, ws) for ws in writer.book.worksheets)
    except FileNotFoundError:
        # file does not exist yet, we will create it
        pass

    if not startrow:
        startrow = 0

    # write out the new sheet
    df.to_excel(writer, sheet_name, startrow=startrow, **to_excel_kwargs)

    # save the workbook
    writer.save()



# Remove duplicate rows from master data
def drop_duplicate_rows(filename):
    '''Function to drop duplicate rows from master dumps'''

    df = pd.read_excel(master_dir+filename, sheet_name= 'Sheet1')
    df.drop_duplicates(keep='first', inplace=True)
    df.to_excel(master_dir+filename, index = False)







In [2]:

#import xlwings as xw
#import numpy as np
import pandas as pd
#from collections import OrderedDict
from time import time
#from dateutil import parser
import os
import Utility_functions


master_dir = 'X:\\Data_Analytics\\ETL\\Master\\'
download_dir = 'X:\\Data_Analytics\\ETL\\Download\\'
output_dir = 'X:\\Data_Analytics\\ETL\\Output\\'


def main_processor(expiry, month, year, file_name,name):
    # if valid Expiry date process and generate reports
    # Remove duplicate rows if any in master files
    Utility_functions.drop_duplicate_rows('RollData.xlsx')
    Utility_functions.drop_duplicate_rows('Indices_dump.xlsx')
    Utility_functions.drop_duplicate_rows('Unrolled_dumped.xlsx')

    # Dataframe with product far, near and rollover values
    data, master = Utility_functions.bhavcopy_processor(file_name)
    result = Utility_functions.sector_rollovers(data)

    # Dump data
    dumper, indices_rollover, unrolled_dumper = Utility_functions.rollover_data_dumper(data, master, result, expiry,
                                                                                       month, year, name)

    # get top and low stocks from grt than avg sheet
    grt_avg_top_stocks, grt_avg_low_stocks = Utility_functions.greaterthanAverage(expiry, month)
    # get top unrolled OI stocks
    unrolled_OI_top_stocks = {}
    if expiry == 'E':
        pass
    else:
        # calculate unrolled OI
        unrolled_OI_top_stocks = Utility_functions.unrolled_OI_Calculator(expiry, month)
    # increased activity
    increased_activity = []
    if expiry in ['E-5', 'E-4', 'E-3']:
        pass
    else:
        # calculate increased activity
        increased_activity = Utility_functions.increased_activity_calc(expiry, month, year)

    # get top 2 leading and lagging sectors
    leading_sectors, lagging_sectors, avg_indices = Utility_functions.leading_lagging_sectors(expiry, month)

    # call final print
    Utility_functions.print_final_report(expiry, avg_indices, increased_activity, leading_sectors, lagging_sectors,
                                         grt_avg_top_stocks, grt_avg_low_stocks, unrolled_OI_top_stocks, name)

    # Update master files
    Utility_functions.append_df_to_excel('RollData.xlsx', df=dumper, header=None, index=False)
    Utility_functions.append_df_to_excel('Indices_dump.xlsx', df=indices_rollover, header=None, index=False)
    Utility_functions.append_df_to_excel('Unrolled_dumped.xlsx', df=unrolled_dumper, header=None, index=False)





def main():

    # traverse through all bhavcopies
    for r, d, f in os.walk(download_dir):
        for csv_file in f:
            file_name = csv_file

            # extract date from input bhavcopy name
            print 'Processing {0} bhavcopy...'.format(file_name)
            name = file_name[2:11]
            date, month, year = name[:2], name[2:5], name[5:]

            # read expiry from master dump to get expiry code
            # if not an expiry throw exception
            try:
                dates = pd.read_excel(master_dir + 'Expiry_dates_master.xlsx')
                expiry = str(dates[(dates['Date'] == int(date))
                                   & (dates['Month'] == month) & (dates['Year'] == int(year))]['Expiry'].values[0])
                main_processor(expiry, month, year, file_name,name)

            except IndexError:
                print 'Exception: {0}/{1}/{2} - Not a valid expiry date...'.format(date, month, year)





start_time = time()

if __name__ == '__main__':
    main()

end_time = time()


print "Execution time: {0} Seconds.... ".format(end_time - start_time)


Processing fo12SEP2018bhav.csv bhavcopy...
Exception: 12/SEP/2018 - Not a valid expiry date...
Processing fo16OCT2018bhav.csv bhavcopy...
Exception: 16/OCT/2018 - Not a valid expiry date...
Processing fo17OCT2018bhav.csv bhavcopy...


Utility_functions.py:20: RuntimeWarning: invalid value encountered in double_scalars
  return round(b / (a + b), 3)


Dumping data for E-5 OCT 2018.....
Processing fo19OCT2018bhav.csv bhavcopy...
Dumping data for E-4 OCT 2018.....
Processing fo22OCT2018bhav.csv bhavcopy...
Dumping data for E-3 OCT 2018.....
Processing fo23OCT2018bhav.csv bhavcopy...
Dumping data for E-2 OCT 2018.....
Processing fo24OCT2018bhav.csv bhavcopy...
Dumping data for E-1 OCT 2018.....
Processing fo25OCT2018bhav.csv bhavcopy...
Dumping data for E OCT 2018.....
Execution time: 214.612999916 Seconds.... 


In [3]:
# Calculate avergae of last three months
def average_calculator(expiry, value, stock_names, data_df, month):
    '''Function to calculate average of last three months'''
    months = {"JAN":0, "FEB":1, "MAR":2, "APR":3, "MAY":4, "JUN":5, "JUL":6, "AUG":7, "SEP":8, "OCT":9, "NOV":10, "DEC":11 }
    #months.keys()[months.values().index(months[month]-1)]
    average = []


    for i in range(len(stock_names)):
        # average.append( round(rolldata.groupby('Expiry').get_group(expiry).loc[stock_names[i]][-3:]['SSF_rollover'].mean(), 2) )
        average.append(round(data_df[(data_df['Expiry']==expiry) &
         ((data_df['Month']==months.keys()[months.values().index(months[month]-1)]) |
        (data_df['Month']==months.keys()[months.values().index(months[month]-2)]) |
        (data_df['Month']==months.keys()[months.values().index(months[month]-3)])) ].loc[stock_names[i]][value].mean(), 2))

    return average

In [6]:
data_df = pd.read_excel(master_dir+'Indices_dump.xlsx')
#data_df.set_index('Indices', inplace=True)
data_df['Date'] = data_df.apply(lambda row: pd.to_datetime(row['Month']+' '+str(row['Year']) ), axis=1)
date = pd.to_datetime(month+" "+year)



if date in dates:
    dates.remove(date)
    


NameError: name 'month' is not defined

In [119]:
data_df.set_index('Indices',inplace=True)
data_df[(data_df['Expiry']=='E-4') &
         ((data_df['Date']==dates[0]) |
        (data_df['Date']==dates[1]) |
        (data_df['Date']==dates[2])) ].loc['Nifty']['Roll'].mean()

0.12833333333333333

In [67]:
data_df[(data_df['Expiry']=='E-4') & (data_df['Indices']=='Nifty') & ]

Expiry Month  Year Indices   Roll       Date
1906    E-4   NOV  2018   Nifty  0.101 2018-11-01
1930    E-4   NOV  2018   Nifty  0.101 2018-11-01
1762    E-4   OCT  2018   Nifty  0.135 2018-10-01
552     E-4   SEP  2018   Nifty  0.100 2018-09-01
528     E-4   AUG  2018   Nifty  0.150 2018-08-01
504     E-4   JUL  2018   Nifty  0.140 2018-07-01
480     E-4   JUN  2018   Nifty  0.070 2018-06-01
456     E-4   MAY  2018   Nifty  0.110 2018-05-01
432     E-4   APR  2018   Nifty  0.100 2018-04-01
408     E-4   MAR  2018   Nifty  0.170 2018-03-01
384     E-4   FEB  2018   Nifty  0.150 2018-02-01
360     E-4   JAN  2018   Nifty  0.090 2018-01-01
336     E-4   DEC  2017   Nifty  0.240 2017-12-01
312     E-4   NOV  2017   Nifty  0.210 2017-11-01
288     E-4   OCT  2017   Nifty  0.070 2017-10-01

In [96]:
# ssf rollovers 
df = pd.read_excel('Report.xlsx', sheet_name='SSF_rollovers')
df['SSF_rollover'] = df['SSF_rollover']*100
df1 = pd.read_excel('Report.xlsx', sheet_name='SSF_avearge')
df1['Average'] = df1['Average']*100
# merge ssf rollovers and average 
df = df.merge(df1, on='SYMBOL', how = 'left')

# indices rollovers
df2 = pd.read_excel('Report.xlsx', sheet_name='Indices_rollovers')
df2['Roll'] = df2['Roll']*100
df3 = pd.read_excel('Report.xlsx', sheet_name='Indices_avearge')
df3['Average'] = df3['Average']*100
# merge 
df2 = df2.merge(df3, on='SYMBOL', how = 'left')

# unrolled rollovers
df4 = pd.read_excel('Report.xlsx', sheet_name='Unrolled_rollovers')

# write dataframe to excel
writer = pd.ExcelWriter('Report.xlsx', engine='openpyxl')

df.to_excel(writer, sheet_name='SSF_rollovers', index=False)
df2.to_excel(writer, sheet_name='Indices_rollovers', index=False)
df4.to_excel(writer, sheet_name='Unrolled_rollovers', index=False)
writer.save()
writer.close()

In [102]:
df

Expiry      SYMBOL BloomCode Month  Year  SSF_rollover  Average
0        E         ACC       ACC   DEC  2018        8490.0     7900
1        E    ADANIENT       ADE   DEC  2018        8700.0     9200
2        E  ADANIPORTS     ADSEZ   DEC  2018        8720.0     8600
3        E  ADANIPOWER     ADANI   DEC  2018        9860.0     9700
4        E  AJANTPHARM       AJP   DEC  2018        9700.0     9700
5        E        ALBK      ALBK   DEC  2018        9500.0     9400
6        E  AMARAJABAT      AMRJ   DEC  2018        8450.0     7800
7        E   AMBUJACEM      ACEM   DEC  2018        8840.0     8600
8        E  APOLLOHOSP      APHS   DEC  2018        7540.0     8300
9        E  APOLLOTYRE      APTY   DEC  2018        8940.0     8100
10       E      ARVIND     ARVND   DEC  2018        8060.0     7900
11       E    ASHOKLEY        AL   DEC  2018        8940.0     9000
12       E  ASIANPAINT      APNT   DEC  2018        7870.0     7300
13       E  AUROPHARMA      ARBP   DEC  2018        8540.0     9100
14       E    AXISBANK      AXSB   DEC  2018        8200.0     8700
15       E  BAJAJ-AUTO     BJAUT   DEC  2018        8750.0     8200
16       E  BAJFINANCE       BAF   DEC  2018        9090.0     8700
17       E  BANKBARODA       BOB   DEC  2018        8090.0     9100
18       E   BANKINDIA       BOI   DEC  2018        8970.0     8800
19       E   BATAINDIA      BATA   DEC  2018        8380.0     7900
20       E         BEL       BHE   DEC  2018        7240.0     8300
21       E        BEML      BEML   DEC  2018        9510.0     9700
22       E   BHARATFIN     BHAFI   DEC  2018        7310.0     8700
23       E  BHARATFORG      BHFC   DEC  2018        9280.0     8900
24       E  BHARTIARTL    BHARTI   DEC  2018        8190.0     8200
25       E        BHEL      BHEL   DEC  2018        7310.0     8100
26       E      BIOCON      BIOS   DEC  2018        8350.0     8900
27       E    BOSCHLTD       BOS   DEC  2018        8760.0     8500
28       E        BPCL      BPCL   DEC  2018        6780.0     7600
29       E   BRITANNIA      BRIT   DEC  2018        8180.0     8900
..     ...         ...       ...   ...   ...           ...      ...
171      E     UJJIVAN   UJJIVAN   DEC  2018        6780.0     8500
172      E      SUZLON      SUEL   DEC  2018        8540.0     8900
173      E         PEL      PIEL   DEC  2018        9030.0     9300
174      E     ESCORTS       ESC   DEC  2018        8740.0     9000
175      E    SHREECEM      SRCM   DEC  2018        5880.0     8700
176      E        MFSL      MAXF   DEC  2018        7880.0     8100
177      E     INDIANB      INBK   DEC  2018        9810.0     9000
178      E  BALKRISIND       BIL   DEC  2018        8730.0     8400
179      E        NBCC      NBCC   DEC  2018        8800.0     8500
180      E         MGL     MAHGL   DEC  2018        7210.0     9600
181      E  BERGEPAINT      BRGR   DEC  2018        9200.0     9300
182      E  CANFINHOME      CANF   DEC  2018        9730.0     8800
183      E    CHOLAFIN      CIFC   DEC  2018        8510.0     8400
184      E        GSFC      GSFC   DEC  2018        9760.0     9500
185      E         MCX       MCX   DEC  2018        9380.0     8900
186      E   NESTLEIND      NEST   DEC  2018        8690.0     8600
187      E      VGUARD      VGRD   DEC  2018        9460.0     9800
188      E  GODFRYPHLP        GP   DEC  2018        9540.0     9400
189      E  BAJAJFINSV     BJFIN   DEC  2018        8510.0     8300
190      E        MRPL      MRPL   DEC  2018        7880.0     8800
191      E     RAYMOND        RW   DEC  2018        8700.0     9200
192      E    RAMCOCEM      TRCL   DEC  2018        8770.0     8100
193      E  NATIONALUM      NACL   DEC  2018        6580.0     8400
194      E     RBLBANK       RBK   DEC  2018        7490.0     8800
195      E  CHENNPETRO       MRL   DEC  2018        8720.0     9400
196      E    SREINFRA      SREI   DEC  2018        8690.0     9200
197      E  MANAPPURAM      MGFL   DEC  2018        7750.0  